In [ ]:
https://www.kaggle.com/scirpus/two-sigma-connect-rental-listing-inquiries/apples-and-oranges-iii/code

In [1]:
import random
import numpy as np
import pandas as pd
from math import exp
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold


def compute_beta(row):
    g = 1.0
    k=5.0
    f=1.0
    cnt = row['cnt'] if row['cnt'] < 200 else float('inf')
    return 1.0 / (g + exp((cnt - k) / f))
        

def calculate_average(sub1, sub2, variable):
    r_k=0.01
    lambda_val = None
    s = pd.DataFrame(data = {
                             variable: sub1.groupby(variable, as_index = False).count()[variable],                              
                             'sumy': sub1.groupby(variable, as_index = False).sum()['y'],
                             'avgY': sub1.groupby(variable, as_index = False).mean()['y'],
                             'cnt': sub1.groupby(variable, as_index = False).count()['y']
                             })

    tmp = sub2.merge(s.reset_index(), how='left', left_on=variable, right_on=variable) 
    del tmp['index']                       
    tmp.loc[pd.isnull(tmp['cnt']), 'cnt'] = 0.0
    tmp.loc[pd.isnull(tmp['cnt']), 'sumy'] = 0.0



    if lambda_val is not None:
        tmp['beta'] = lambda_val
    else:
        tmp['beta'] = tmp.apply(compute_beta, axis = 1)

    tmp['adj_avg'] = tmp.apply(lambda row: (1.0 - row['beta']) * row['avgY'] + row['beta'] * row['pred_0'],
                               axis = 1)

    tmp.loc[pd.isnull(tmp['avgY']), 'avgY'] = tmp.loc[pd.isnull(tmp['avgY']), 'pred_0']
    tmp.loc[pd.isnull(tmp['adj_avg']), 'adj_avg'] = tmp.loc[pd.isnull(tmp['adj_avg']), 'pred_0']
    tmp['random'] = np.random.uniform(size = len(tmp))
    tmp['adj_avg'] = tmp.apply(lambda row: row['adj_avg'] *(1 + (row['random'] - 0.5) * r_k),
                               axis = 1)

    return tmp['adj_avg'].ravel()
    

def categorical_average(variable, y, pred_0, feature_name):
    #cv for training set 
    k_fold = StratifiedKFold(5, random_state=myseed)
    X_train[feature_name] = -999 
    for (train_index, cv_index) in k_fold.split(np.zeros(len(X_train)),
                                                X_train['interest_level'].ravel()):
        sub = pd.DataFrame(data = {variable: X_train[variable],
                                   'y': X_train[y],
                                   'pred_0': X_train[pred_0]})
            
        sub1 = sub.iloc[train_index]        
        sub2 = sub.iloc[cv_index]
        
        X_train.loc[cv_index, feature_name] = calculate_average(sub1, sub2, variable)
    
    #for test set
    sub1 = pd.DataFrame(data = {variable: X_train[variable],
                                'y': X_train[y],
                                'pred_0': X_train[pred_0]})
    sub2 = pd.DataFrame(data = {variable: X_test[variable],
                                'y': X_test[y],
                                'pred_0': X_test[pred_0]})
    X_test.loc[:, feature_name] = calculate_average(sub1, sub2, variable)                               


def transform_data(X):
    #add features    
    feat_sparse = feature_transform.transform(X["features"])
    vocabulary = feature_transform.vocabulary_
    del X['features']
    X1 = pd.DataFrame([ pd.Series(feat_sparse[i].toarray().ravel()) for i in np.arange(feat_sparse.shape[0]) ])
    X1.columns = list(sorted(vocabulary.keys()))
    X = pd.concat([X.reset_index(), X1.reset_index()], axis = 1)
    del X['index']
    
    X["num_photos"] = X["photos"].apply(len)
    X['created'] = pd.to_datetime(X["created"])
    X["num_description_words"] = X["description"].apply(lambda x: len(x.split(" ")))
    X['price_per_bed'] = X['price'] / X['bedrooms']    
    X['price_per_bath'] = X['price'] / X['bathrooms']
    X['price_per_room'] = X['price'] / (X['bathrooms'] + X['bedrooms'] )
    
    X['low'] = 0
    X.loc[X['interest_level'] == 0, 'low'] = 1
    X['medium'] = 0
    X.loc[X['interest_level'] == 1, 'medium'] = 1
    X['high'] = 0
    X.loc[X['interest_level'] == 2, 'high'] = 1
    
    X['display_address'] = X['display_address'].apply(lambda x: x.lower().strip())
    X['street_address'] = X['street_address'].apply(lambda x: x.lower().strip())
    
    X['pred0_low'] = low_count * 1.0 / train_size
    X['pred0_medium'] = medium_count * 1.0 / train_size
    X['pred0_high'] = high_count * 1.0 / train_size
    
    X.loc[X['manager_id'].isin(managers_with_one_lot['manager_id'].ravel()), 
          'manager_id'] = "-1"
    X.loc[X['building_id'].isin(buildings_with_one_lot['building_id'].ravel()), 
          'building_id'] = "-1"
    X.loc[X['display_address'].isin(addresses_with_one_lot['display_address'].ravel()), 
          'display_address'] = "-1"
          
    return X


def normalize_high_cardinality_data():
    high_cardinality = ["building_id", "manager_id"]
    for c in high_cardinality:
        categorical_average(c, "medium", "pred0_medium", c + "_mean_medium")
        categorical_average(c, "high", "pred0_high", c + "_mean_high")

        
def transform_categorical_data():
    categorical = ['building_id', 'manager_id', 
                   'display_address', 'street_address']
                   
    for f in categorical:
        encoder = LabelEncoder()
        encoder.fit(list(X_train[f]) + list(X_test[f])) 
        X_train[f] = encoder.transform(X_train[f].ravel())
        X_test[f] = encoder.transform(X_test[f].ravel())
                  

def remove_columns(X):
    columns = ["photos", "pred0_high", "pred0_low", "pred0_medium",
               "description", "low", "medium", "high",
               "interest_level", "created", 
               'display_address', 'street_address']
    for c in columns:
        del X[c]

        
def find_objects_with_only_one_record(feature_name):
    temp = pd.concat([X_train[feature_name].reset_index(), 
                      X_test[feature_name].reset_index()])
    temp = temp.groupby(feature_name, as_index = False).count()
    return temp[temp['index'] == 1]


def Outputs(p):
    return 1.0/(1.0+np.exp(-p))


def GPZero(data):
    v = pd.DataFrame()
    v["i0"] = np.tanh(((data["manager_level_low"] + (-(((data["manager_level_high"] - ((data["building_id_mean_medium"] < data["price"]).astype(float))) - ((((data["manager_level_low"] < data["manager_level_low"]).astype(float)) < data["price_per_bed"]).astype(float)))))) + np.tanh((data["price"] + np.tanh((data["price"] + (((data["residents_lounge"] < data["price"]).astype(float)) + ((data["price_per_bed"] < (data["price"] * 2.0)).astype(float)))))))))
    v["i1"] = np.tanh((-(((data["manager_id_mean_medium"] + (data["building_id_mean_high"] - ((((data["price"] * 2.0) * 2.0) * 2.0) + (((data["private_outdoor_space"] < data["price"]).astype(float)) - ((data["no_fee"] + (((data["no_fee"] + (((data["reduced_fee"] - data["longitude"]) - data["longitude"]) - data["longitude"]))/2.0) - ((data["reduced_fee"] < data["price"]).astype(float))))/2.0)))))/2.0))))
    v["i2"] = np.tanh(((((((data["furnished"] < data["manager_level_medium"]).astype(float)) < data["price_per_bath"]).astype(float)) + (-(((data["bedrooms"] + (data["furnished"] + ((((data["private"] - data["furnished"]) > data["manager_level_high"]).astype(float)) * (data["building_id_mean_medium"] + (((data["_pets_ok_"] > data["building_id_mean_medium"]).astype(float)) * (data["manager_level_medium"] + (data["building_id_mean_medium"] + data["hardwood_floors"])))))))/2.0))))/2.0))
    v["i3"] = np.tanh(((((((((data["prewar"] + ((data["building_id_mean_high"] + data["listing_id"])/2.0)) + (((data["prewar"] * data["prewar"]) < data["latitude"]).astype(float))) + data["bedrooms"])/2.0) + (((((data["deck"] + data["price_per_bed"]) * 2.0) + ((data["manager_id_mean_medium"] > ((data["highrise"] < data["latitude"]).astype(float))).astype(float)))/2.0) * 2.0))/2.0) + ((data["price_per_room"] * 2.0) * 2.0))/2.0))
    v["i4"] = np.tanh((data["price"] + ((-(((data["air_conditioning"] < (data["common_outdoor_space"] - data["price"])).astype(float)))) + ((((((data["common_outdoor_space"] + data["hardwood"])/2.0) > (((data["outdoor_areas"] + ((data["num_photos"] + data["common_outdoor_space"])/2.0))/2.0) / 2.0)).astype(float)) + (-((((data["laundry_in_unit"] - ((data["manager_level_medium"] + data["outdoor_areas"])/2.0)) / 2.0) / 2.0))))/2.0))))
    v["i5"] = np.tanh((data["wifi_access"] * (((-((np.tanh((data["building_id_mean_high"] * ((data["laundry_in_building"] + data["building_id_mean_medium"]) * 2.0))) * 2.0))) + (data["exclusive"] + (data["renovated"] + data["_photos"]))) + ((data["granite_kitchen"] + (data["laundry_in_building"] - np.tanh((data["building_id_mean_medium"] * ((data["manager_level_medium"] * 2.0) * 2.0))))) * 2.0))))
    v["i6"] = np.tanh((data["site_garage"] + np.tanh((data["price_per_room"] + ((((data["longitude"] * (31.006277 * (((data["doorman"] - np.tanh(((data["longitude"] > (data["24"] / 2.0)).astype(float)))) < np.tanh((data["longitude"] * 31.006277))).astype(float)))) / 2.0) + np.tanh(((data["longitude"] > (data["24"] / 2.0)).astype(float))))/2.0)))))
    v["i7"] = np.tanh(((((((data["price_per_room"] + data["price_per_bed"]) + data["dryer"])/2.0) + data["price_per_bed"]) * ((data["post_war"] < ((((data["no_fee"] + data["price_per_bed"])/2.0) * (data["price_per_room"] * (2.0))) / 2.0)).astype(float))) + (((((-(((2.0) / 2.0))) + data["bathrooms"])/2.0) < (-(((data["bathrooms"] < data["price_per_bed"]).astype(float))))).astype(float))))
    v["i8"] = np.tanh((data["shares_ok"] - (((data["roof"] - (((((((((-(((data["num_photos"] + data["stainless_steel_appliances"])/2.0))) > (1.570796 / 2.0)).astype(float)) * 2.0) > (((1.570796 / 2.0) - ((data["listing_id"] + data["roof"])/2.0)) / 2.0)).astype(float)) * 2.0) * 2.0) * 2.0)) + (((data["roof"] > data["price"]).astype(float)) / 2.0))/2.0)))
    v["i9"] = np.tanh(((((3.0) + (data["listing_id"] / 2.0)) > (((((((3.0) > data["bedrooms"]).astype(float)) > (((data["num_photos"] + (0.636620 * 2.0)) > (((((float(31.006277 > (3.0))) > (data["manager_level_low"] + (data["_dishwasher_"] - ((data["building_id_mean_medium"] + 0.636620)/2.0)))).astype(float)) * 2.0) * 2.0)).astype(float))).astype(float)) * 2.0) * 2.0)).astype(float)))
    v["i10"] = np.tanh((data["childrens_playroom"] * (data["multi"] + ((data["exclusive"] + ((data["furnished"] - (((data["manager_level_low"] < data["exclusive"]).astype(float)) * 2.0)) - data["manager_level_high"])) + (data["manager_level_low"] + (((((data["building_id_mean_medium"] < data["prewar"]).astype(float)) + (((data["price_per_bed"] > ((data["furnished"] > data["no_fee"]).astype(float))).astype(float)) * 2.0)) * 2.0) * 2.0))))))
    v["i11"] = np.tanh(((((data["latitude"] < ((data["post"] + (data["_pets_ok_"] + data["post"]))/2.0)).astype(float)) + (((-(((data["longitude"] < (data["_pets_ok_"] - data["latitude"])).astype(float)))) + ((data["_pets_ok_"] + data["price_per_room"]) + (((data["price_per_bed"] > (data["latitude"] - ((data["price_per_bed"] > (data["_pets_ok_"] - data["latitude"])).astype(float)))).astype(float)) / 2.0)))/2.0))/2.0))
    v["i12"] = np.tanh(((((data["price"] + data["price"]) + data["price"]) * ((data["price"] < data["site_laundry"]).astype(float))) * (((((data["num_description_words"] * (data["outdoor_entertainment_space"] - ((((data["_dryer"] + data["loft"])/2.0) < data["price_per_room"]).astype(float)))) + data["loft"])/2.0) < (data["price_per_room"] * ((data["price_per_room"] * data["bathrooms"]) + data["subway"]))).astype(float))))
    v["i13"] = np.tanh(((((data["longitude"] > ((((data["lowrise"] * 2.0) * data["building_id_mean_medium"]) > np.tanh(((data["manager_level_medium"] + (data["manager_level_high"] - ((((((data["longitude"] * data["building_id_mean_high"]) > (((data["lowrise"] * data["longitude"]) + (data["bedrooms"] * 2.0))/2.0)).astype(float)) > ((data["manager_level_high"] + data["building_id"])/2.0)).astype(float)) / 2.0)))/2.0))).astype(float))).astype(float)) / 2.0) / 2.0))
    v["i14"] = np.tanh(((data["price_per_room"] + ((data["residents_garden"] * (data["dogs_allowed"] * ((((((-(data["price_per_room"])) < ((((data["residents_garden"] * data["bedrooms"]) + ((((data["bedrooms"] * 2.0) > data["dogs_allowed"]).astype(float)) - data["bedrooms"]))/2.0) / 2.0)).astype(float)) * 2.0) * 2.0) - data["no_fee"]))) * 2.0))/2.0))
    v["i15"] = np.tanh(np.tanh((((data["latitude"] + ((data["price"] + (data["price_per_room"] - ((((((data["laundry_in_building"] > data["price"]).astype(float)) > ((data["price_per_room"] + data["latitude"])/2.0)).astype(float)) > data["latitude"]).astype(float))))/2.0))/2.0) * ((data["dryer"] > ((data["price"] + ((data["price"] - data["latitude"]) * np.tanh(((data["doorman"] > data["price"]).astype(float)))))/2.0)).astype(float)))))
    v["i16"] = np.tanh(((np.tanh(((data["bathrooms"] < (data["bathrooms"] * (data["bathrooms"] - (data["listing_id"] - ((3.141593 + data["longitude"]) + (data["residents_garden"] * (3.141593 - (((1.0) + data["bathrooms"]) - (data["subway"] * 2.0))))))))).astype(float))) > (((((1.0) + data["longitude"])/2.0) + data["bathrooms"])/2.0)).astype(float)))
    v["i17"] = np.tanh((((((data["num_photos"] < ((((data["highrise"] + (data["green_building"] + data["roof"]))/2.0) + (-(((data["bedrooms"] + (((-(data["listing_id"])) < (((data["bathrooms"] < ((data["listing_id"] + ((((data["green_building"] < 3.141593).astype(float)) + (-(data["num_photos"])))/2.0))/2.0)).astype(float)) * 2.0)).astype(float)))/2.0))))/2.0)).astype(float)) / 2.0) / 2.0) / 2.0))
    v["i18"] = np.tanh(np.tanh(((((((data["longitude"] * 2.0) * 2.0) * (((data["residents_garden"] < data["latitude"]).astype(float)) + (data["latitude"] * 2.0))) * ((((data["residents_garden"] < data["latitude"]).astype(float)) > data["longitude"]).astype(float))) * 2.0) * ((((data["video_intercom"] < (-(np.tanh(data["latitude"])))).astype(float)) > ((data["price_per_room"] > (-(data["residents_garden"]))).astype(float))).astype(float)))))
    v["i19"] = np.tanh((((data["price_per_bath"] * (((((((data["price_per_bath"] * 2.0) > data["terrace"]).astype(float)) > data["bedrooms"]).astype(float)) > ((data["building_id_mean_medium"] + (data["live"] + ((data["terrace"] > data["new_construction"]).astype(float))))/2.0)).astype(float))) - (data["dryer"] * ((data["building_id_mean_medium"] > ((((data["manager_level_low"] > data["garage"]).astype(float)) / 2.0) / 2.0)).astype(float)))) * 2.0))
    v["i20"] = np.tanh((((((data["price"] + (data["price"] * (((data["listing_id"] + data["doorman"]) + data["site_super"])/2.0)))/2.0) + (data["post"] * ((((data["listing_id"] + ((np.tanh((-(data["doorman"]))) > data["building_id_mean_high"]).astype(float))) + (data["listing_id"] + data["doorman"])) * 2.0) * ((data["doorman"] > data["listing_id"]).astype(float)))))/2.0) * 2.0))
    v["i21"] = np.tanh(((((data["pool"] * 2.0) * ((((data["price_per_bed"] + data["residents_garden"])/2.0) > ((((data["building"] > (data["eat_in_kitchen"] * (((data["bedrooms"] / 2.0) + data["building"])/2.0))).astype(float)) + data["laundry_room"])/2.0)).astype(float))) * ((data["building"] > (((data["residents_lounge"] + ((data["price_per_bed"] + data["residents_lounge"])/2.0))/2.0) * (data["bedrooms"] / 2.0))).astype(float))) * 2.0))
    v["i22"] = np.tanh(((((data["indoor_pool"] * 2.0) * (((data["high_ceiling"] - data["bathrooms"]) - data["bathrooms"]) - data["building_id_mean_high"])) + (data["manager_id_mean_high"] * (-(((data["manager_level_medium"] < np.tanh(np.tanh((-(((data["high_ceilings"] < ((data["indoor_pool"] * 2.0) * (data["live"] - data["bathrooms"]))).astype(float))))))).astype(float)))))) / 2.0))
    v["i23"] = np.tanh((-((((np.tanh((data["building_id_mean_high"] * data["outdoor_entertainment_space"])) + (((data["building_id_mean_high"] * ((data["simplex"] > np.tanh((data["building_id_mean_high"] * data["outdoor_entertainment_space"]))).astype(float))) + (data["building_id"] * ((data["simplex"] > ((data["building_id_mean_medium"] * ((data["simplex"] > (data["building_id"] * (data["furnished"] / 2.0))).astype(float))) / 2.0)).astype(float))))/2.0))/2.0) * 2.0))))
    v["i24"] = np.tanh(((0.636620 < (((data["private_backyard"] - (data["longitude"] * data["reduced_fee"])) + (((data["bathrooms"] * data["bathrooms"]) + data["highrise"])/2.0)) - (((((((data["private_backyard"] - data["private_backyard"]) * data["longitude"]) - (data["listing_id"] * 0.636620)) + data["highrise"])/2.0) + data["bathrooms"])/2.0))).astype(float)))
    v["i25"] = np.tanh(np.tanh((((((data["parking_space"] + (data["manager_id_mean_high"] + data["no_fee"])) + ((data["building_id_mean_medium"] + data["private_outdoor_space"]) + (data["bedrooms"] + (data["private_outdoor_space"] + data["terrace"]))))/2.0) * (data["parking_space"] + (((data["new_construction"] < data["latitude"]).astype(float)) * 2.0))) * (data["outdoor_areas"] * ((data["air_conditioning"] < data["latitude"]).astype(float))))))
    v["i26"] = np.tanh(np.tanh(((np.tanh(np.tanh((data["dryer_in_unit"] / 2.0))) + (((np.tanh(np.tanh(data["common_outdoor_space"])) / 2.0) + (((data["high_speed_internet"] > ((((((((data["manager_level_high"] + data["laundry_in_unit"]) * data["manager_level_high"]) + data["wheelchair_access"])/2.0) / 2.0) - (np.tanh(data["manager_level_high"]) + data["laundry_in_unit"])) * 2.0) * 2.0)).astype(float)) / 2.0))/2.0))/2.0)))
    v["i27"] = np.tanh((data["roof"] * ((data["price_per_bed"] > ((-((data["roof"] + (((data["granite_kitchen"] + (data["building_id_mean_medium"] + ((np.tanh((data["price_per_bed"] + (((data["granite_kitchen"] < data["price_per_bed"]).astype(float)) * data["building_id_mean_medium"]))) > data["building_id_mean_medium"]).astype(float)))) > ((data["price_per_bed"] > data["common"]).astype(float))).astype(float))))) * ((data["manager_level_low"] > data["building_id_mean_medium"]).astype(float)))).astype(float))))
    v["i28"] = np.tanh(((-((((data["short_term_allowed"] * ((np.tanh(data["short_term_allowed"]) > (((data["price_per_bed"] < ((data["price"] > (data["exposed_brick"] * (-(((data["exposed_brick"] < data["laundry_in_building"]).astype(float)))))).astype(float))).astype(float)) * ((-(((data["exposed_brick"] < data["bedrooms"]).astype(float)))) + data["bedrooms"]))).astype(float))) * 2.0) * ((data["time_doorman"] < data["price_per_bed"]).astype(float))))) * 2.0))
    v["i29"] = np.tanh((data["no_fee"] * (((data["actual_apt"] + np.tanh((data["duplex"] + (data["in_super"] - data["price_per_bath"])))) + (((((data["hi_rise"] < (data["short_term_allowed"] + data["longitude"])).astype(float)) / 2.0) + data["midrise"])/2.0)) * ((data["longitude"] > (data["in_super"] - (data["level"] - ((data["price_per_bath"] + data["manager_id_mean_medium"])/2.0)))).astype(float)))))
    v["i30"] = np.tanh((-(((data["price_per_bath"] * (data["dogs_allowed"] - (((((data["subway"] + ((data["no_fee"] > ((data["virtual_doorman"] + data["num_photos"])/2.0)).astype(float)))/2.0) > data["num_photos"]).astype(float)) - (data["num_photos"] - ((data["loft"] > (((data["patio"] / 2.0) + data["laundry_in_building"])/2.0)).astype(float)))))) * ((data["no_fee"] > (data["num_photos"] - data["no_fee"])).astype(float))))))
    v["i31"] = np.tanh(((data["fitness_center"] + ((data["in_superintendent"] * data["bedrooms"]) + (((data["furnished"] < data["num_photos"]).astype(float)) + (((data["residents_lounge"] < data["dishwasher"]).astype(float)) + (data["furnished"] - ((data["listing_id"] + ((-(data["bedrooms"])) * 2.0))/2.0)))))) * (data["residents_lounge"] * ((data["longitude"] < (data["in_superintendent"] * data["no_fee"])).astype(float)))))
    v["i32"] = np.tanh((((data["childrens_playroom"] * 2.0) * 2.0) * (-(((data["longitude"] > ((((((-((data["childrens_playroom"] * 2.0))) > (((data["common_outdoor_space"] * 2.0) * 2.0) * 2.0)).astype(float)) > ((data["manager_level_medium"] > ((((data["building_id_mean_high"] + 0.636620)/2.0) + np.tanh(((data["manager_level_medium"] + data["building_id_mean_high"])/2.0)))/2.0)).astype(float))).astype(float)) * (data["building_id_mean_high"] * 2.0))).astype(float))))))
    v["i33"] = np.tanh(((data["price"] + data["price"]) * ((data["price_per_room"] < (data["common_outdoor_space"] + ((data["manager_level_high"] < (data["common_outdoor_space"] + ((data["common_outdoor_space"] + data["manager_level_low"]) + ((-(((data["manager_level_low"] * (data["building_id_mean_medium"] - data["manager_level_low"])) * data["building_id_mean_medium"]))) - (data["post_war"] + data["building_id_mean_medium"]))))).astype(float)))).astype(float))))
    v["i34"] = np.tanh(((((data["sublet"] > ((data["longitude"] + ((((((data["building"] + data["site_garage"]) * 2.0) + data["common_outdoor_space"]) * 2.0) < data["num_description_words"]).astype(float)))/2.0)).astype(float)) > (((data["bathrooms"] + ((data["site_garage"] < (0.318310 + ((data["building"] + data["latitude"]) * 2.0))).astype(float)))/2.0) / 2.0)).astype(float)))
    v["i35"] = np.tanh(np.tanh(np.tanh((-(((data["price"] + ((data["stainless_steel_appliances"] > ((data["price"] + (data["price"] * np.tanh(((data["price"] + ((data["bedrooms"] * (((data["price"] * ((data["building_id_mean_high"] > (data["washer_in_unit"] + data["price"])).astype(float))) > data["garage"]).astype(float))) / 2.0))/2.0))))/2.0)).astype(float)))/2.0))))))
    v["i36"] = np.tanh((((((data["stainless_steel_appliances"] > (data["building_id_mean_medium"] * (data["site_garage"] * (data["latitude"] - (data["war"] / 2.0))))).astype(float)) + ((data["site_garage"] > ((data["latitude"] * ((((data["latitude"] / 2.0) * data["bedrooms"]) > data["latitude"]).astype(float))) - (data["latitude"] / 2.0))).astype(float))) / 2.0) / 2.0))
    v["i37"] = np.tanh(((((data["view"] * ((data["latitude"] > (-((data["actual_apt"] * ((data["longitude"] > (-((data["latitude"] * 2.0)))).astype(float)))))).astype(float))) * 2.0) * 2.0) * (((((-((data["latitude"] * 2.0))) > (((data["hardwood_floors"] + (-(data["price_per_bed"]))) / 2.0) / 2.0)).astype(float)) > (-(data["price_per_bed"]))).astype(float))))
    v["i38"] = np.tanh((-((data["building_id_mean_high"] * ((((((np.tanh(data["reduced_fee"]) + ((data["luxury_building"] + ((data["no_fee"] + (((data["outdoor_space"] - data["building_id"]) + (((((data["outdoor_space"] + data["building_id_mean_high"])/2.0) + (data["cats_allowed"] - data["laundry_in_unit"]))/2.0) - data["laundry_in_unit"]))/2.0))/2.0))/2.0))/2.0) + (-(data["latitude"])))/2.0) + data["wifi_access"])/2.0)))))
    v["i39"] = np.tanh(((data["roof_deck"] * 2.0) * (data["longitude"] * (-(((((data["longitude"] / 2.0) > data["post_war"]).astype(float)) - ((data["longitude"] > ((data["bedrooms"] + (((data["dishwasher"] - data["loft"]) > ((data["roof_deck"] * ((data["_dishwasher_"] < data["longitude"]).astype(float))) * 2.0)).astype(float)))/2.0)).astype(float))))))))
    v["i40"] = np.tanh((((-1.0 - (-(((data["roofdeck"] < (((data["price"] / 2.0) + ((data["building_id_mean_high"] < (((data["num_photos"] * np.tanh(data["price"])) < (data["num_description_words"] * ((data["num_photos"] + data["exclusive"])/2.0))).astype(float))).astype(float)))/2.0)).astype(float))))) + (data["price_per_room"] * ((data["roofdeck"] > (data["price"] / 2.0)).astype(float))))/2.0))
    v["i41"] = np.tanh((-(((data["_photos"] * (((((data["longitude"] > (data["newly_renovated"] * (data["lowrise"] * data["dishwasher"]))).astype(float)) > (((((data["longitude"] + (data["_dryer"] / 2.0)) > ((((data["longitude"] > data["latitude"]).astype(float)) / 2.0) / 2.0)).astype(float)) > (data["common_terrace"] * (data["roof_deck"] + data["longitude"]))).astype(float))).astype(float)) * 2.0)) * 2.0))))
    v["i42"] = np.tanh((data["price"] * (((((data["site_laundry"] < data["price"]).astype(float)) * ((-(((data["dryer"] > data["price"]).astype(float)))) * 2.0)) > (((data["pool"] < data["listing_id"]).astype(float)) + ((-((((((((data["price"] + ((data["longitude"] < data["price"]).astype(float)))/2.0) + data["longitude"])/2.0) * 2.0) < data["view"]).astype(float)))) - data["listing_id"]))).astype(float))))
    v["i43"] = np.tanh((-((data["price_per_bed"] * np.tanh(((((data["price_per_room"] - data["wheelchair_access"]) - (((data["outdoor_space"] > data["price_per_room"]).astype(float)) + (data["price_per_bed"] + (((data["price_per_room"] < ((data["outdoor_space"] > data["manager_id_mean_high"]).astype(float))).astype(float)) + data["stainless_steel_appliances"])))) + ((data["garden"] + (((data["stainless_steel_appliances"] < data["price_per_room"]).astype(float)) * data["pre"]))/2.0))/2.0))))))
    v["i44"] = np.tanh((-((((((data["roofdeck"] + data["site_super"])/2.0) * 2.0) * 2.0) * (((-(((data["marble_bath"] < data["longitude"]).astype(float)))) > (data["building_id"] + (((data["num_description_words"] * ((data["building_id"] + ((((data["swimming_pool"] < data["roofdeck"]).astype(float)) + (data["washer_in_unit"] * 2.0))/2.0))/2.0)) * 2.0) * 2.0))).astype(float))))))
    v["i45"] = np.tanh(np.tanh(np.tanh(np.tanh(np.tanh(((data["longitude"] * ((((data["num_photos"] * (((-(data["longitude"])) < data["price_per_room"]).astype(float))) + ((data["num_photos"] + (data["num_photos"] * (-((((data["num_photos"] + data["air_conditioning"])/2.0) - data["new_construction"])))))/2.0)) > (-((data["longitude"] - data["new_construction"])))).astype(float))) * 2.0))))))
    v["i46"] = np.tanh((data["longitude"] * ((data["war"] + (((-((((((data["wheelchair_ramp"] + data["storage"]) * data["central_ac"]) + ((data["price"] < ((data["wheelchair_ramp"] < data["longitude"]).astype(float))).astype(float))) + (data["hardwood_floors"] + data["washer_"])) + data["wheelchair_ramp"]))) + data["storage"]) + ((data["furnished"] + data["elevator"])/2.0))) / 2.0)))
    v["i47"] = np.tanh((((np.tanh(data["laundry"]) > (data["latitude"] * data["cats_allowed"])).astype(float)) * ((data["price"] - (data["exposed_brick"] - (data["price"] - data["renovated"]))) * ((((data["price"] > ((data["exposed_brick"] - ((data["laundry"] > (data["price"] / 2.0)).astype(float))) * 2.0)).astype(float)) * 2.0) * 2.0))))
    v["i48"] = np.tanh((data["price_per_room"] * (((((((data["manager_id_mean_medium"] * data["manager_id_mean_medium"]) * 2.0) + data["manager_id_mean_medium"]) * 2.0) + data["concierge"]) * data["work"]) + (((data["price_per_room"] > ((data["manager_level_low"] < (data["doorman"] / 2.0)).astype(float))).astype(float)) + ((data["bedrooms"] > (((data["manager_level_low"] < ((data["renovated"] < data["price_per_room"]).astype(float))).astype(float)) * 2.0)).astype(float))))))
    v["i49"] = np.tanh((((((data["central_ac"] * data["concierge"]) < data["dryer"]).astype(float)) + (((((data["simplex"] > (data["building_id_mean_high"] * data["building_id_mean_high"])).astype(float)) > ((data["exposed_brick"] < (-((((data["building_id_mean_medium"] + ((data["central_ac"] < data["building_id_mean_high"]).astype(float))) < (data["building_id_mean_high"] * ((((data["simplex"] > data["concierge"]).astype(float)) < data["building_id_mean_high"]).astype(float)))).astype(float))))).astype(float))).astype(float)) * 2.0))/2.0))
    v["i50"] = np.tanh(((((data["valet_parking"] < (data["valet_parking"] + (data["valet_parking"] - ((data["valet_parking"] * ((data["bedrooms"] + ((data["bedrooms"] + (((((data["bedrooms"] + data["space"])/2.0) + ((data["virtual_doorman"] + (data["eat_in_kitchen"] / 2.0))/2.0))/2.0) / 2.0))/2.0))/2.0)) / 2.0)))).astype(float)) * 2.0) * 2.0))
    v["i51"] = np.tanh((data["laundry_in_building"] * (((data["outdoor_entertainment_space"] * ((((6.0) - (data["listing_id"] * data["laundry_in_building"])) - (data["doorman"] * 2.0)) - (data["light"] * 2.0))) / 2.0) * ((data["price_per_bed"] > (((data["laundry_in_building"] < (data["latitude"] * data["laundry_in_building"])).astype(float)) * (data["latitude"] * data["doorman"]))).astype(float)))))
    v["i52"] = np.tanh(((((data["business_center"] * 2.0) * 2.0) * 2.0) * (((data["childrens_playroom"] - (-((data["price"] * 2.0)))) + (((data["price"] + (-(((-(data["courtyard"])) * ((data["fireplace"] + ((data["courtyard"] < ((data["bedrooms"] + ((data["childrens_playroom"] < data["childrens_playroom"]).astype(float)))/2.0)).astype(float)))/2.0))))) < (data["childrens_playroom"] * 2.0)).astype(float)))/2.0)))
    v["i53"] = np.tanh((((data["_photos"] * (((((data["duplex"] + data["manager_id_mean_high"]) + data["pre"]) > data["_photos"]).astype(float)) - (((((data["common_roof_deck"] + (data["duplex"] + data["manager_id_mean_high"])) + data["pre"]) + np.tanh(data["pre"])) + data["common_roof_deck"]) * data["laundry_in_unit"]))) + (-(np.tanh(data["health_club"]))))/2.0))
    v["i54"] = np.tanh(((np.tanh(data["live"]) + (-(((data["longitude"] > (((((data["dishwasher"] > (data["parking"] / 2.0)).astype(float)) * 2.0) < (data["num_photos"] * ((((data["num_photos"] * data["microwave"]) / 2.0) > data["parking"]).astype(float)))).astype(float))).astype(float))))) * ((data["num_photos"] * ((data["doorman"] > data["parking_space"]).astype(float))) / 2.0)))
    v["i55"] = np.tanh(((data["actual_apt"] * (-(((data["latitude"] < ((data["longitude"] + ((data["longitude"] + (((((data["_dishwasher_"] + np.tanh(data["latitude"]))/2.0) * 2.0) > (((data["hardwood_floors"] < (((data["hardwood_floors"] < (data["dishwasher"] + data["longitude"])).astype(float)) + data["latitude"])).astype(float)) * 2.0)).astype(float)))/2.0))/2.0)).astype(float))))) * 2.0))
    v["i56"] = np.tanh(((data["longitude"] + (data["brownstone"] * (data["microwave"] + (data["num_description_words"] + (((((((data["manager_id_mean_medium"] * data["manager_level_high"]) * data["manager_level_high"]) > (data["manager_id_mean_medium"] * data["manager_level_high"])).astype(float)) * 2.0) - (((data["manager_id_mean_medium"] + data["manager_level_high"])/2.0) * (data["num_description_words"] + (data["num_description_words"] + data["reduced_fee"])))) * 2.0)))))/2.0))
    v["i57"] = np.tanh(((data["loft"] * (data["common_terrace"] * ((data["manager_id"] < (((data["manager_level_low"] + (((((data["building_id"] < (data["manager_level_low"] / 2.0)).astype(float)) > data["manager_level_low"]).astype(float)) / 2.0)) + data["laundry_in_building"])/2.0)).astype(float)))) * 2.0))
    v["i58"] = np.tanh((((data["war"] > data["num_description_words"]).astype(float)) * (data["exclusive"] * (((((data["space"] * ((((data["exclusive"] < data["new_construction"]).astype(float)) > (data["num_description_words"] * ((data["fitness_center"] > (data["fitness_center"] / 2.0)).astype(float)))).astype(float))) < (((((data["war"] * data["num_description_words"]) * 2.0) * 2.0) < data["war"]).astype(float))).astype(float)) * data["new_construction"]) / 2.0))))
    v["i59"] = np.tanh(np.tanh(np.tanh((data["private_outdoor_space"] * ((data["stainless_steel_appliances"] > np.tanh(((data["price_per_room"] + ((((data["price"] + data["central_a"])/2.0) / 2.0) * np.tanh((((((data["private_outdoor_space"] > (((((data["courtyard"] * np.tanh(data["roofdeck"])) / 2.0) / 2.0) + data["central_a"])/2.0)).astype(float)) * 2.0) > data["longitude"]).astype(float)))))/2.0))).astype(float))))))
    v["i60"] = np.tanh((((np.tanh(((data["latitude"] < (data["longitude"] / 2.0)).astype(float))) + ((-(((data["latitude"] < ((((data["lounge"] - ((data["residents_garden"] > (((data["longitude"] - ((data["residents_garden"] > (data["price_per_bath"] / 2.0)).astype(float))) - data["latitude"]) / 2.0)).astype(float))) - data["latitude"]) * data["furnished"]) / 2.0)).astype(float)))) / 2.0))/2.0) / 2.0))
    v["i61"] = np.tanh(np.tanh(np.tanh(np.tanh(np.tanh(np.tanh((((((data["parking_space"] < (data["latitude"] * 2.0)).astype(float)) > (data["no_fee"] * ((data["bathrooms"] < (-(data["common_outdoor_space"]))).astype(float)))).astype(float)) * (np.tanh(np.tanh(np.tanh(np.tanh((data["latitude"] * 2.0))))) * ((data["parking_space"] < ((data["latitude"] + data["roof"])/2.0)).astype(float))))))))))
    v["i62"] = np.tanh(((((((1.86890649795532227) < (data["listing_id"] + ((-(data["live"])) + ((-(data["live"])) + ((((data["building_id_mean_high"] + data["subway"]) + data["marble_bath"])/2.0) * (-(data["bike_room"]))))))).astype(float)) > ((((data["building_id_mean_high"] + ((data["exclusive"] < data["luxury_building"]).astype(float)))/2.0) + ((data["washer_in_unit"] < data["live"]).astype(float)))/2.0)).astype(float)) / 2.0))
    v["i63"] = np.tanh(((((data["price_per_room"] - np.tanh(((data["price_per_room"] + ((((data["reduced_fee"] + ((data["high_ceilings"] > data["price"]).astype(float)))/2.0) > (data["fitness_center"] * ((data["prewar"] > data["price_per_room"]).astype(float)))).astype(float)))/2.0))) / 2.0) + (np.tanh(((data["elevator"] < data["high_ceilings"]).astype(float))) * ((data["price_per_room"] + (((data["exposed_brick"] < data["price_per_room"]).astype(float)) / 2.0))/2.0)))/2.0))
    v["i64"] = np.tanh((data["outdoor_areas"] * (data["balcony"] * (((((data["roof_deck"] > (-(data["bathrooms"]))).astype(float)) < ((data["listing_id"] > data["roofdeck"]).astype(float))).astype(float)) + ((data["s_kitchen_"] + (((data["roofdeck"] > data["dogs_allowed"]).astype(float)) * data["loft"]))/2.0)))))
    v["i65"] = np.tanh(np.tanh(((data["bathrooms"] < (((((data["reduced_fee"] * ((data["parking_space"] > ((data["site_parking"] + (data["fitness"] * data["business_center"]))/2.0)).astype(float))) / 2.0) * ((data["longitude"] > (data["24"] * 2.0)).astype(float))) + ((data["work"] * (data["longitude"] * 2.0)) - ((data["fitness"] < ((data["longitude"] + data["indoor_pool"])/2.0)).astype(float))))/2.0)).astype(float))))
    v["i66"] = np.tanh(np.tanh((data["parking_space"] * (np.tanh((-(((((data["sauna"] + data["children"])/2.0) > (data["building_id_mean_high"] * ((((((((data["manager_level_medium"] * data["pets_on_approval"]) * (data["building_id_mean_high"] - data["children"])) * data["building_id_mean_high"]) + data["laundry"])/2.0) * data["space"]) + data["children"])/2.0))).astype(float))))) * 2.0))))
    v["i67"] = np.tanh((-(((data["live_in_super"] * 2.0) * ((data["outdoor_areas"] > ((data["latitude"] * ((data["longitude"] > ((((((data["manager_id"] + ((data["manager_id"] + data["dogs_allowed"])/2.0))/2.0) + data["manager_level_low"])/2.0) * ((data["longitude"] > data["dogs_allowed"]).astype(float))) * ((data["outdoor_areas"] > data["live_in_super"]).astype(float)))).astype(float))) * ((data["outdoor_areas"] > data["private_outdoor_space"]).astype(float)))).astype(float))))))
    v["i68"] = np.tanh((data["site_garage"] * (((data["space"] + (data["building_id_mean_medium"] * data["short_term_allowed"])) < (-(((data["manager_id_mean_high"] < ((data["multi"] + (-(((((data["pre"] > (data["building_id_mean_high"] + (data["building_id_mean_medium"] * ((np.tanh(data["fitness_center"]) < data["multi"]).astype(float))))).astype(float)) * data["fitness_center"]) * data["fitness_center"]))))/2.0)).astype(float))))).astype(float))))
    v["i69"] = np.tanh((-((data["pets_on_approval"] * (((((-((data["laundry_in_building"] * ((((data["dining_room"] - (data["outdoor_space"] + (data["dogs_allowed"] * data["air_conditioning"]))) + (-((data["num_description_words"] * ((data["swimming_pool"] - (data["private_balcony"] / 2.0)) / 2.0))))) + data["dogs_allowed"])/2.0)))) / 2.0) / 2.0) > data["luxury_building"]).astype(float))))))
    v["i70"] = np.tanh(((((data["price"] + (data["price"] + data["price"]))/2.0) * ((data["building_id_mean_high"] < (data["price"] + data["fitness_center"])).astype(float))) * ((data["outdoor_space"] < (data["price"] * (-(((((data["building_id_mean_medium"] < (-(data["outdoor_space"]))).astype(float)) + ((data["price"] < data["building_id_mean_high"]).astype(float))) + data["fitness_center"]))))).astype(float))))
    v["i71"] = np.tanh((data["price_per_room"] * ((np.tanh(data["num_description_words"]) > (np.tanh(np.tanh(((-(((data["price_per_room"] < (((data["building_id_mean_medium"] < (((data["manager_id_mean_medium"] + data["bathrooms"]) > (data["building_id_mean_medium"] * 2.0)).astype(float))).astype(float)) + data["building_id_mean_medium"])).astype(float)))) / 2.0))) + (((3.0) > data["bathrooms"]).astype(float)))).astype(float))))
    v["i72"] = np.tanh((np.tanh((np.tanh(data["bike_room"]) * (data["duplex"] - ((((data["price_per_bed"] > np.tanh(data["bike_room"])).astype(float)) * 2.0) * ((data["manager_level_low"] * data["exposed_brick"]) + ((data["price_per_bed"] < ((data["manager_level_low"] * data["exposed_brick"]) * ((data["price_per_bed"] > (data["manager_level_low"] * np.tanh(data["duplex"]))).astype(float)))).astype(float))))))) / 2.0))
    v["i73"] = np.tanh(((-(data["roofdeck"])) * (data["manager_level_high"] - (((((data["longitude"] > (((data["laundry_in_unit"] * data["building_id_mean_medium"]) < data["roofdeck"]).astype(float))).astype(float)) < ((data["building_id_mean_medium"] < ((data["latitude"] < ((data["latitude"] < (data["air_conditioning"] * data["laundry_in_unit"])).astype(float))).astype(float))).astype(float))).astype(float)) - ((data["longitude"] > (data["live"] * (-(data["roofdeck"])))).astype(float))))))
    v["i74"] = np.tanh((-((data["furnished"] * (data["valet_parking"] * (data["listing_id"] - np.tanh((((((data["dining_room"] - (data["building_id_mean_high"] - data["manager_id_mean_medium"])) - data["manager_level_high"]) < ((data["valet_parking"] < data["listing_id"]).astype(float))).astype(float)) - ((data["building_id_mean_high"] * data["building_id_mean_high"]) - (data["dining_room"] - (data["manager_level_high"] * data["building_id_mean_high"])))))))))))
    v["i75"] = np.tanh(((data["price_per_bath"] + data["price_per_bath"]) * (-(((data["manager_level_medium"] < np.tanh(((((data["loft"] + (data["wheelchair_access"] / 2.0))/2.0) - (((((data["_dryer"] > data["price_per_bath"]).astype(float)) * ((data["manager_id_mean_medium"] < data["loft"]).astype(float))) + data["virtual_doorman"])/2.0)) - (((((data["manager_id_mean_medium"] / 2.0) > data["_dryer"]).astype(float)) + data["_dryer"])/2.0)))).astype(float))))))
    v["i76"] = np.tanh((data["price"] * ((data["newly_renovated"] > (data["price"] * (((data["washer"] > ((data["dogs_allowed"] - ((data["war"] < data["storage"]).astype(float))) - (((((-(((data["war"] < data["fitness_center"]).astype(float)))) > data["live_in_super"]).astype(float)) - data["manager_id"]) / 2.0))).astype(float)) / 2.0))).astype(float))))
    v["i77"] = np.tanh(((data["deck"] + np.tanh(((((data["pets_on_approval"] + -1.0)/2.0) + -1.0)/2.0))) * (((((((-(data["post"])) + data["furnished"])/2.0) + data["furnished"])/2.0) < (-(((data["24"] < (data["common_terrace"] * (-1.0 * ((((data["pets_on_approval"] > data["bedrooms"]).astype(float)) - data["bathrooms"]) / 2.0)))).astype(float))))).astype(float))))
    v["i78"] = np.tanh(((data["backyard"] * (data["building_id"] * (((((data["balcony"] * 2.0) - data["dryer"]) - data["backyard"]) - ((data["laundry_in_building"] < data["prewar"]).astype(float))) * (data["prewar"] - ((((data["laundry_in_building"] + ((data["balcony"] < ((data["space"] + data["dryer"])/2.0)).astype(float)))/2.0) > data["private_terrace"]).astype(float)))))) / 2.0))
    v["i79"] = np.tanh(((data["common_outdoor_space"] * data["site_parking"]) * (data["manager_id"] + ((data["time_doorman"] + (data["time_doorman"] * (((((data["time_doorman"] < data["common_outdoor_space"]).astype(float)) + (data["time_doorman"] * (((data["courtyard"] < ((data["time_doorman"] < data["site_parking"]).astype(float))).astype(float)) - data["balcony"]))) * ((data["listing_id"] < data["parking_space"]).astype(float))) - data["site_parking"])))/2.0))))
    v["i80"] = np.tanh((data["dryer_in_building"] * ((data["health_club"] - np.tanh(((data["war"] + (data["health_club"] + (((((data["building_id_mean_medium"] > data["price"]).astype(float)) * 2.0) - data["bedrooms"]) * 2.0))) * 2.0))) - (((data["hardwood_floors"] + data["green_building"])/2.0) - data["dishwasher"]))))
    v["i81"] = np.tanh(((((((data["manager_level_high"] + np.tanh(np.tanh(data["building_id_mean_high"])))/2.0) < (((data["indoor_pool"] > data["price_per_bath"]).astype(float)) / 2.0)).astype(float)) * (((((data["virtual_doorman"] + data["dining_room"])/2.0) < (((data["manager_level_high"] + np.tanh(np.tanh(np.tanh(data["building_id_mean_high"]))))/2.0) / 2.0)).astype(float)) * data["price_per_bath"])) * ((data["virtual_doorman"] < (data["price_per_bath"] / 2.0)).astype(float))))
    v["i82"] = np.tanh((data["lowrise"] * ((data["common_roof_deck"] > (data["manager_id_mean_high"] * ((data["manager_level_high"] + (np.tanh(((data["site_garage"] + data["private_outdoor_space"])/2.0)) - (((data["no_fee"] * np.tanh(((data["manager_id_mean_high"] < np.tanh((data["no_fee"] * data["manager_level_high"]))).astype(float)))) + ((data["luxury_building"] > (data["manager_id_mean_high"] * data["washer"])).astype(float)))/2.0)))/2.0))).astype(float))))
    v["i83"] = np.tanh((np.tanh(np.tanh(np.tanh(np.tanh((data["work"] * (data["longitude"] - (((-(((np.tanh(((data["stainless_steel_appliances"] < data["price_per_room"]).astype(float))) / 2.0) + ((((data["work"] > data["work"]).astype(float)) + ((data["longitude"] + ((data["roof"] < data["latitude"]).astype(float)))/2.0))/2.0)))) > (data["num_photos"] * data["cats_allowed"])).astype(float)))))))) * 2.0))
    v["i84"] = np.tanh((((data["longitude"] + (-(data["price"]))) + (data["pool"] * ((data["price_per_bath"] > (data["bedrooms"] * np.tanh((-(data["price_per_bath"]))))).astype(float)))) * ((((data["elevator"] > np.tanh((data["bedrooms"] - (data["pool"] - (data["outdoor_space"] * data["price"]))))).astype(float)) < (data["wheelchair_ramp"] - data["longitude"])).astype(float))))
    v["i85"] = np.tanh((data["price"] * (((((((((data["price_per_bed"] < (data["no_fee"] * ((data["air_conditioning"] + data["price"])/2.0))).astype(float)) < (data["manager_level_low"] + ((data["latitude"] + ((data["manager_level_low"] + (-(data["dining_room"]))) + np.tanh(((data["dishwasher"] > data["price_per_bed"]).astype(float)))))/2.0))).astype(float)) * 2.0) * 2.0) + data["no_fee"])/2.0) / 2.0)))
    v["i86"] = np.tanh((data["site_garage"] * ((-(((((data["longitude"] > ((data["longitude"] < ((data["building_id_mean_medium"] + data["site_parking_lot"])/2.0)).astype(float))).astype(float)) > (-(((((data["bathrooms"] * data["multi"]) / 2.0) > ((((data["newly_renovated"] > (-(np.tanh(data["building_id_mean_medium"])))).astype(float)) + ((((data["building_id_mean_medium"] + data["longitude"])/2.0) < data["site_garage"]).astype(float)))/2.0)).astype(float))))).astype(float)))) * 2.0)))
    v["i87"] = np.tanh(((data["indoor_pool"] * ((data["manager_id_mean_high"] + (((((np.tanh(data["bedrooms"]) + data["manager_id_mean_high"]) + data["listing_id"])/2.0) - data["listing_id"]) - (data["manager_id_mean_medium"] * (np.tanh(data["manager_id_mean_high"]) * ((data["listing_id"] * data["listing_id"]) * data["listing_id"])))))/2.0)) * (((data["manager_id_mean_high"] + data["listing_id"])/2.0) + data["listing_id"])))
    v["i88"] = np.tanh((data["latitude"] * ((-((((((data["price"] > data["private_outdoor_space"]).astype(float)) > ((data["private_outdoor_space"] + data["hardwood_floors"]) + data["latitude"])).astype(float)) + ((data["laundry_in_unit"] > ((data["private_outdoor_space"] + data["hardwood_floors"]) + data["building_id_mean_medium"])).astype(float))))) * ((data["laundry_in_unit"] > ((data["manager_id"] + (data["building_id_mean_medium"] + np.tanh(data["latitude"])))/2.0)).astype(float)))))
    v["i89"] = np.tanh((((((-(data["balcony"])) * (data["parking"] * data["private_outdoor_space"])) + (((data["private_terrace"] + data["parking"])/2.0) / 2.0))/2.0) * np.tanh((-((data["pets_on_approval"] * (data["balcony"] - (data["deck"] + data["private_outdoor_space"]))))))))
    v["i90"] = np.tanh((data["pool"] * ((data["manager_level_low"] > (((data["manager_level_low"] + data["pool"]) + ((data["manager_level_low"] > ((((((data["no_fee"] + ((data["bike_room"] > (data["dining_room"] - data["no_fee"])).astype(float)))/2.0) + data["laundry_in_building"])/2.0) / 2.0) + (-((data["dishwasher"] / 2.0))))).astype(float)))/2.0)).astype(float))))
    v["i91"] = np.tanh((data["garage"] * (-(((data["residents_lounge"] * 2.0) * ((data["building_id_mean_high"] > ((((((data["residents_lounge"] * 2.0) > data["bedrooms"]).astype(float)) > (data["private_outdoor_space"] - data["price_per_room"])).astype(float)) - ((data["bathrooms"] > ((data["outdoor_space"] < data["private_outdoor_space"]).astype(float))).astype(float)))).astype(float)))))))
    v["i92"] = np.tanh((-(((data["deck"] * ((data["_pets_ok_"] > ((-(((data["latitude"] + (-((((-(((data["latitude"] + ((data["latitude"] + (-(((data["latitude"] + data["microwave"])/2.0))))/2.0))/2.0))) < data["longitude"]).astype(float)))))/2.0))) * 2.0)).astype(float))) * 2.0))))
    v["i93"] = np.tanh(((data["site_garage"] * 2.0) * ((((data["swimming_pool"] + (((data["fitness_center"] - ((data["wheelchair_access"] * data["laundry_in_unit"]) * 2.0)) - data["building_id_mean_high"]) - (((data["swimming_pool"] > (data["building_id_mean_high"] - data["wheelchair_access"])).astype(float)) * 2.0)))/2.0) > ((data["roof_deck"] < ((data["no_pets"] + data["building_id_mean_high"])/2.0)).astype(float))).astype(float))))
    v["i94"] = np.tanh((data["indoor_pool"] * (((data["elevator"] < data["building_id_mean_medium"]).astype(float)) + (data["num_photos"] + (-((((((((-((((-(data["building_id"])) < ((data["num_photos"] < ((data["building_id"] < data["num_photos"]).astype(float))).astype(float))).astype(float)))) < data["building_id"]).astype(float)) * 2.0) < (data["building_id_mean_medium"] + ((data["price_per_bath"] * 2.0) + 1.570796))).astype(float)) * 2.0)))))))
    v["i95"] = np.tanh((np.tanh((np.tanh((data["virtual_doorman"] - (data["short_term_allowed"] * data["price_per_room"]))) - ((data["common_parking"] + np.tanh((-((data["bedrooms"] * (((data["simplex"] - (-((data["bedrooms"] * data["latitude"])))) - (data["fitness"] - ((data["fitness"] > data["latitude"]).astype(float)))) / 2.0))))))/2.0))) / 2.0))
    v["i96"] = np.tanh((((data["price_per_room"] + ((data["price_per_bed"] + ((data["longitude"] > (-((((data["price"] + ((((data["central_a"] * data["doorman"]) * 2.0) / 2.0) + (((data["manager_id"] < ((data["price_per_bed"] * data["central_a"]) * 2.0)).astype(float)) * data["longitude"])))/2.0) / 2.0)))).astype(float)))/2.0))/2.0) / 2.0))
    v["i97"] = np.tanh(((data["24"] * ((data["manager_id_mean_medium"] > ((data["doorman"] > (-(((data["latitude"] > (-((data["outdoor_areas"] * data["building_id_mean_medium"])))).astype(float))))).astype(float))).astype(float))) * ((((((data["outdoor_areas"] < ((data["latitude"] > data["virtual_doorman"]).astype(float))).astype(float)) + (((np.tanh(data["num_description_words"]) > data["doorman"]).astype(float)) * data["num_description_words"]))/2.0) * 2.0) * 2.0)))
    v["i98"] = np.tanh((data["residents_lounge"] * (data["listing_id"] * (((data["manager_id"] < np.tanh(((data["listing_id"] < data["manager_id_mean_medium"]).astype(float)))).astype(float)) - ((data["manager_id_mean_medium"] * (((data["no_fee"] * (((data["listing_id"] / 2.0) * 2.0) / 2.0)) + (data["listing_id"] - data["no_fee"]))/2.0)) + (((-(data["latitude"])) > data["playroom"]).astype(float)))))))
    v["i99"] = np.tanh(np.tanh(np.tanh(np.tanh(np.tanh(np.tanh((-((data["pet_friendly"] * (data["wifi_access"] - np.tanh((((data["bedrooms"] - ((data["num_description_words"] - ((data["manager_id"] - (data["war"] + (data["num_description_words"] + data["listing_id"]))) * 2.0)) + (data["num_description_words"] + data["live"]))) * 2.0) * 2.0))))))))))))
    return Outputs(v.sum(axis=1))

def GPOne(data):
    v = pd.DataFrame()
    v["i0"] = np.tanh((np.tanh(np.tanh(np.tanh(data["building_id_mean_medium"]))) + (np.tanh(data["manager_level_medium"]) - ((((((((((((1.570796 + ((data["manager_level_medium"] < (-(np.tanh(data["building_id_mean_medium"])))).astype(float)))/2.0) * 2.0) + (((data["manager_level_medium"] < data["price"]).astype(float)) + data["price"]))/2.0) * 2.0) + data["price_per_room"])/2.0) * 2.0) + data["price"])/2.0) * 2.0))))
    v["i1"] = np.tanh(((((-(((data["price_per_room"] > ((((((data["manager_level_medium"] + (np.tanh(data["building_id_mean_high"]) + data["manager_level_high"])) + (((data["no_fee"] + data["manager_level_medium"]) + ((data["manager_level_medium"] * data["manager_level_high"]) + np.tanh(data["building_id_mean_medium"])))/2.0))/2.0) / 2.0) - data["price_per_bed"]) - data["price"])).astype(float)))) * 2.0) * 2.0) * 2.0))
    v["i2"] = np.tanh((((((-(data["manager_level_medium"])) > ((data["building_id_mean_medium"] > data["price"]).astype(float))).astype(float)) - data["manager_level_medium"]) * ((((((data["price"] > (-(data["building_id_mean_medium"]))).astype(float)) > data["building_id_mean_medium"]).astype(float)) - data["manager_id_mean_high"]) * (data["manager_level_medium"] * ((data["manager_id_mean_medium"] < (-((((data["building_id_mean_medium"] - data["price"]) > np.tanh(data["building_id_mean_medium"])).astype(float))))).astype(float))))))
    v["i3"] = np.tanh((-(((((data["price"] + ((((data["price_per_room"] + data["price"])/2.0) < data["simplex"]).astype(float))) + ((data["in_super"] + ((((data["price"] < np.tanh((data["washer_in_unit"] - (-(data["basement_storage"]))))).astype(float)) < data["price_per_room"]).astype(float)))/2.0))/2.0) + (((data["price"] - (-(((data["num_photos"] < -1.0).astype(float))))) + data["price_per_room"])/2.0)))))
    v["i4"] = np.tanh(np.tanh((-(np.tanh((data["longitude"] - ((data["furnished"] + (((data["longitude"] + ((((data["longitude"] < (data["work"] / 2.0)).astype(float)) + np.tanh(((data["private_outdoor_space"] + (data["laundry_in_unit"] + (data["manager_level_high"] + ((data["longitude"] + data["no_fee"]) - data["building_id_mean_medium"]))))/2.0))) + data["longitude"]))/2.0) / 2.0))/2.0)))))))
    v["i5"] = np.tanh((-(((data["price_per_bed"] + np.tanh(((((data["building_id_mean_medium"] < (-(((data["concierge"] < data["eat_in_kitchen"]).astype(float))))).astype(float)) + ((data["manager_level_medium"] + np.tanh(((((data["concierge"] - ((data["renovated"] > data["price"]).astype(float))) - data["reduced_fee"]) - ((data["renovated"] > data["price"]).astype(float))) - data["manager_level_high"])))/2.0))/2.0)))/2.0))))
    v["i6"] = np.tanh((data["indoor_pool"] * ((data["listing_id"] * (((data["latitude"] + (data["listing_id"] * data["listing_id"]))/2.0) * data["listing_id"])) + ((data["outdoor_areas"] - ((data["manager_id_mean_medium"] - data["building_id_mean_medium"]) * (data["manager_id_mean_medium"] - data["building_id_mean_medium"]))) - ((data["laundry_in_building"] - data["listing_id"]) - (data["building_id_mean_medium"] - data["indoor_pool"]))))))
    v["i7"] = np.tanh(((((-(((data["washer_in_unit"] > ((data["price_per_bath"] - (((((data["washer_in_unit"] < ((data["price_per_bed"] + data["bathrooms"])/2.0)).astype(float)) + (-((data["common"] / 2.0))))/2.0) / 2.0)) * (((data["bathrooms"] < ((data["concierge"] < ((data["price_per_bed"] + ((data["washer_in_unit"] < data["latitude"]).astype(float)))/2.0)).astype(float))).astype(float)) / 2.0))).astype(float)))) * 2.0) * 2.0) * 2.0))
    v["i8"] = np.tanh((data["price_per_room"] * (np.tanh((((data["manager_level_high"] / 2.0) + (data["manager_level_medium"] + ((data["exposed_brick"] - ((((data["new_construction"] < data["price_per_room"]).astype(float)) / 2.0) * 2.0)) - ((data["price_per_room"] > ((data["no_fee"] + (((((((data["new_construction"] < data["price_per_room"]).astype(float)) / 2.0) < data["loft"]).astype(float)) < data["manager_level_medium"]).astype(float)))/2.0)).astype(float)))))/2.0)) * 2.0)))
    v["i9"] = np.tanh((data["lowrise"] * ((np.tanh(data["building_id_mean_medium"]) - (((((((data["building_id_mean_medium"] / 2.0) > (data["building_id_mean_high"] - data["common"])).astype(float)) > (data["manager_id_mean_high"] + data["building_id_mean_medium"])).astype(float)) < data["price"]).astype(float))) - np.tanh((data["building_id_mean_medium"] - np.tanh(data["building_id_mean_medium"]))))))
    v["i10"] = np.tanh(((-(((((((data["_pets_ok_"] + 0.318310)/2.0) + (data["longitude"] * data["longitude"]))/2.0) > (((((((data["longitude"] - data["bedrooms"]) + (0.318310 - data["wifi_access"]))/2.0) + 1.0)/2.0) > ((((data["listing_id"] + 0.318310)/2.0) > ((1.0 > (data["longitude"] * 1.0)).astype(float))).astype(float))).astype(float))).astype(float)))) * 2.0))
    v["i11"] = np.tanh((-((data["price"] * ((((data["price"] < (data["private_balcony"] * 2.0)).astype(float)) > (data["building_id_mean_medium"] + (-((data["manager_level_low"] * (data["building_id_mean_medium"] + (data["building_id_mean_medium"] + (-((data["manager_level_low"] * (((((data["price"] < ((data["private_balcony"] * 2.0) * 2.0)).astype(float)) + data["price"]) > data["private_balcony"]).astype(float)))))))))))).astype(float))))))
    v["i12"] = np.tanh((data["price_per_bath"] * (((data["building_id_mean_high"] * (data["manager_level_medium"] + (data["ft_doorman"] - data["manager_level_low"]))) + ((data["furnished"] + (((data["price"] > data["price_per_bath"]).astype(float)) - (-((((data["furnished"] > data["price"]).astype(float)) * (data["manager_level_medium"] + (data["dishwasher"] - data["manager_level_low"]))))))) - data["site_super"])) / 2.0)))
    v["i13"] = np.tanh(np.tanh(np.tanh(np.tanh(np.tanh(np.tanh((data["reduced_fee"] * (((data["latitude"] < data["_photos"]).astype(float)) + ((data["_photos"] < ((data["dryer"] + (data["price_per_bed"] + ((data["latitude"] + np.tanh(((-(((data["price_per_bed"] > ((data["latitude"] > ((data["no_fee"] > data["_photos"]).astype(float))).astype(float))).astype(float)))) / 2.0)))/2.0)))/2.0)).astype(float))))))))))
    v["i14"] = np.tanh((data["residents_lounge"] * (-(((((((data["residents_lounge"] > (data["price_per_room"] * (((data["residents_lounge"] * 2.0) > (data["price_per_room"] * ((((np.tanh(data["simplex"]) + data["no_fee"]) * 2.0) < (data["residents_lounge"] / 2.0)).astype(float)))).astype(float)))).astype(float)) * 2.0) * 2.0) + data["dishwasher"]) + data["no_fee"])))))
    v["i15"] = np.tanh(((data["in_super"] - (-(data["concierge"]))) * ((((data["speed_internet"] - (data["longitude"] + ((((data["latitude"] < data["actual_apt"]).astype(float)) > ((data["latitude"] < data["longitude"]).astype(float))).astype(float)))) / 2.0) < (data["longitude"] + ((((data["virtual_doorman"] < data["longitude"]).astype(float)) < (data["latitude"] + data["common_terrace"])).astype(float)))).astype(float))))
    v["i16"] = np.tanh((data["deck"] * ((data["walk"] - ((((((data["latitude"] > (-(data["playroom"]))).astype(float)) + np.tanh(((data["latitude"] < data["longitude"]).astype(float))))/2.0) < (-((data["longitude"] - (((data["granite_kitchen"] - ((data["latitude"] < data["latitude"]).astype(float))) - ((data["latitude"] < data["luxury_building"]).astype(float))) / 2.0))))).astype(float))) * 2.0)))
    v["i17"] = np.tanh((((data["ft_doorman"] * (np.tanh((((-(((((data["new_construction"] > (((-(data["ft_doorman"])) + (-(data["building_id_mean_medium"])))/2.0)).astype(float)) < data["building_id_mean_high"]).astype(float)))) - ((data["bedrooms"] < data["ft_doorman"]).astype(float))) + (-((data["num_photos"] + data["ft_doorman"]))))) + ((data["num_description_words"] < data["new_construction"]).astype(float)))) * 2.0) * 2.0))
    v["i18"] = np.tanh(np.tanh(((np.tanh(data["private_outdoor_space"]) + ((((data["high_ceiling"] > (data["price_per_bed"] + data["price_per_bath"])).astype(float)) + data["storage"])/2.0)) * ((((data["high_ceiling"] * (((data["high_ceiling"] * (((data["bathrooms"] < data["high_ceiling"]).astype(float)) / 2.0)) < data["price_per_bed"]).astype(float))) < np.tanh(data["high_ceiling"])).astype(float)) / 2.0))))
    v["i19"] = np.tanh((((data["view"] + ((((data["view"] + (data["simplex"] - data["microwave"]))/2.0) + data["simplex"])/2.0))/2.0) * (data["_dishwasher_"] + (-(((data["building_id_mean_medium"] < (((data["laundry_in_building"] - ((data["simplex"] < data["_dishwasher_"]).astype(float))) - (data["view"] * data["simplex"])) - data["view"])).astype(float)))))))
    v["i20"] = np.tanh((data["furnished"] * (np.tanh(np.tanh(data["dryer_in_unit"])) * ((((data["no_fee"] + (((data["live_in_super"] * data["live_in_super"]) + (data["manager_id_mean_high"] - data["dryer_in_unit"])) + (data["live_in_super"] - data["building_id"])))/2.0) + (data["manager_id_mean_medium"] + np.tanh((data["building_id_mean_high"] - ((data["price_per_room"] > data["live_in_super"]).astype(float))))))/2.0))))
    v["i21"] = np.tanh(np.tanh(np.tanh((((((data["roofdeck"] * 2.0) * data["num_photos"]) * data["num_photos"]) + (data["price_per_room"] * (np.tanh(np.tanh((data["reduced_fee"] + np.tanh(np.tanh(np.tanh((data["num_photos"] + (-(((data["building_id"] > (-((((data["building_id_mean_medium"] / 2.0) > data["roof_deck"]).astype(float))))).astype(float))))))))))) * 2.0)))/2.0))))
    v["i22"] = np.tanh(((((data["in_superintendent"] + np.tanh((data["s_playroom"] + data["in_superintendent"]))) + data["granite_kitchen"]) * (((((data["manager_id_mean_high"] > ((data["price"] > data["basement_storage"]).astype(float))).astype(float)) + data["price_per_bed"]) < data["price"]).astype(float))) * (((data["hi_rise"] + ((((data["wheelchair_ramp"] + data["price"])/2.0) + data["wheelchair_ramp"])/2.0)) < data["price_per_bed"]).astype(float))))
    v["i23"] = np.tanh((-(((data["childrens_playroom"] + ((((((data["private_balcony"] + ((-1.0 < (((data["latitude"] > ((data["manager_level_medium"] > ((data["childrens_playroom"] < data["bathrooms"]).astype(float))).astype(float))).astype(float)) - data["bedrooms"])).astype(float)))/2.0) < data["bathrooms"]).astype(float)) < (((data["bathrooms"] - (data["fitness"] * (data["listing_id"] * data["listing_id"]))) - data["exclusive"]) / 2.0)).astype(float)))/2.0))))
    v["i24"] = np.tanh((np.tanh(((data["manager_id_mean_high"] + np.tanh((data["manager_id_mean_high"] + data["building_id_mean_high"]))) * np.tanh(((data["longitude"] * (data["building_id_mean_high"] + np.tanh((np.tanh((data["garden"] * (data["manager_id_mean_high"] + data["building_id_mean_high"]))) * data["building_id_mean_high"])))) * (data["manager_id_mean_high"] + (data["manager_id_mean_high"] + data["building_id_mean_medium"])))))) / 2.0))
    v["i25"] = np.tanh((data["reduced_fee"] * (-(((data["level"] > ((data["price"] + ((data["reduced_fee"] * (((-(data["manager_id_mean_high"])) > (-(((data["price"] > data["reduced_fee"]).astype(float))))).astype(float))) - ((((data["sauna"] + ((data["price"] + data["common_outdoor_space"])/2.0))/2.0) > (data["reduced_fee"] * (-(data["garden"])))).astype(float))))/2.0)).astype(float))))))
    v["i26"] = np.tanh((data["publicoutdoor"] * (((-((((-(((data["longitude"] > data["price_per_bed"]).astype(float)))) > data["manager_level_medium"]).astype(float)))) - (-((((-(((data["roof"] > data["price_per_bed"]).astype(float)))) > (-(data["longitude"]))).astype(float))))) - (-((((-(((np.tanh(data["roof"]) > data["price"]).astype(float)))) > (-(data["longitude"]))).astype(float)))))))
    v["i27"] = np.tanh(((data["24"] * 2.0) * (np.tanh(((((np.tanh((data["bedrooms"] * data["no_fee"])) > data["bedrooms"]).astype(float)) - data["no_fee"]) + (data["bedrooms"] * (data["bedrooms"] * (data["bedrooms"] * -1.0))))) + np.tanh(((data["no_fee"] * (data["bedrooms"] * data["no_fee"])) * data["no_fee"])))))
    v["i28"] = np.tanh((data["num_photos"] * ((((((data["sauna"] + (-(data["deck"]))) + (data["common_terrace"] * (-(data["duplex"])))) + (-(data["residents_lounge"]))) + (data["private_outdoor_space"] * data["latitude"]))/2.0) * ((data["latitude"] > ((((-(data["listing_id"])) < ((-(data["deck"])) / 2.0)).astype(float)) * data["deck"])).astype(float)))))
    v["i29"] = np.tanh((((-(((-1.0 > (((data["parking_space"] + (data["price_per_room"] * 2.0)) + (-(((data["roofdeck"] > ((-(((data["price_per_room"] > ((data["parking_space"] > ((data["ft_doorman"] * 2.0) * 2.0)).astype(float))).astype(float)))) * 2.0)).astype(float))))) * 2.0)).astype(float)))) * ((data["manager_id_mean_high"] < ((data["price_per_room"] < data["parking_space"]).astype(float))).astype(float))) * (8.0)))
    v["i30"] = np.tanh((data["price_per_bath"] * (((data["24"] > data["longitude"]).astype(float)) - ((np.tanh(np.tanh(data["doorman"])) > (((data["private_outdoor_space"] > ((((data["listing_id"] > ((data["manager_id_mean_high"] > ((data["longitude"] > data["price_per_bath"]).astype(float))).astype(float))).astype(float)) + ((data["price_per_bath"] + np.tanh(np.tanh(data["doorman"])))/2.0))/2.0)).astype(float)) - ((data["longitude"] > data["view"]).astype(float)))).astype(float)))))
    v["i31"] = np.tanh((((((data["residents_lounge"] - (-((data["newly_renovated"] * ((np.tanh(data["laundry_in_building"]) > ((((data["price"] / 2.0) / 2.0) - data["building_id"]) / 2.0)).astype(float)))))) > ((data["price"] / 2.0) / 2.0)).astype(float)) > ((data["building_id"] * ((data["backyard"] + ((data["actual_apt"] + (-(data["midrise"])))/2.0))/2.0)) - data["roofdeck"])).astype(float)))
    v["i32"] = np.tanh(((data["private_backyard"] * ((data["exposed_brick"] + ((data["in_superintendent"] + (data["sauna"] + data["publicoutdoor"]))/2.0))/2.0)) * ((data["num_photos"] - ((data["publicoutdoor"] + (((-(data["manager_level_low"])) + data["backyard"])/2.0))/2.0)) - (data["sauna"] * ((data["manager_level_low"] > data["publicoutdoor"]).astype(float))))))
    v["i33"] = np.tanh((data["price_per_bed"] * (np.tanh(np.tanh(data["latitude"])) + (((data["patio"] > (-(data["parking_space"]))).astype(float)) + (data["price_per_bed"] + (data["24"] + (-(((data["price_per_bed"] > np.tanh(((((data["high_speed_internet"] > (-(((data["price"] > (data["parking_space"] * data["price_per_bed"])).astype(float))))).astype(float)) < data["price_per_bed"]).astype(float)))).astype(float))))))))))
    v["i34"] = np.tanh((((((data["green_building"] * 2.0) - data["price"]) * 2.0) * 2.0) * (((-(data["longitude"])) < (data["storage"] - (data["green_building"] * ((data["storage"] > (((data["price"] > data["washer"]).astype(float)) * (((data["doorman"] - data["manager_level_low"]) - data["price"]) - data["green_building"]))).astype(float))))).astype(float))))
    v["i35"] = np.tanh((data["price_per_bed"] * np.tanh((((data["price_per_room"] + data["sublet"]) * 2.0) - (-((((data["num_description_words"] + (data["dogs_allowed"] * (((((((((data["price_per_bed"] > data["num_description_words"]).astype(float)) < data["price_per_room"]).astype(float)) < (data["price_per_bed"] / 2.0)).astype(float)) + (-(data["price_per_room"]))) * 2.0) * 2.0)))/2.0) / 2.0)))))))
    v["i36"] = np.tanh((data["childrens_playroom"] * ((data["laundry_in_unit"] * 2.0) * ((((data["laundry_in_unit"] * (data["childrens_playroom"] - data["latitude"])) > ((data["latitude"] > (data["latitude"] * (data["manager_id_mean_high"] + data["manager_id_mean_high"]))).astype(float))).astype(float)) - ((((data["price_per_room"] > (((data["laundry_in_unit"] - data["latitude"]) > data["laundry_in_unit"]).astype(float))).astype(float)) > data["manager_id_mean_high"]).astype(float))))))
    v["i37"] = np.tanh((data["in_super"] * ((-(((((data["building_id"] - ((data["bedrooms"] > data["in_super"]).astype(float))) - (-(((data["terrace"] + (data["building_id"] - data["in_super"]))/2.0)))) - data["laundry_in_unit"]) / 2.0))) * ((data["dishwasher"] > (-(((((data["dishwasher"] > (data["pre"] - data["highrise"])).astype(float)) + data["pre"])/2.0)))).astype(float)))))
    v["i38"] = np.tanh(np.tanh((data["building_id_mean_medium"] * (data["midrise"] * (((data["balcony"] + ((((data["midrise"] - data["listing_id"]) + data["elevator"]) + (data["latitude"] - data["listing_id"])) + (((data["actual_apt"] * data["_photos"]) * data["actual_apt"]) + data["parking_space"]))) + (data["building_id"] - data["pre"]))/2.0)))))
    v["i39"] = np.tanh((((-1.0 > (data["high_ceilings"] + np.tanh(data["num_photos"]))).astype(float)) * (-((data["price"] + ((data["price"] + ((data["high_ceilings"] > np.tanh(data["manager_level_high"])).astype(float))) + (data["high_ceilings"] + (data["price"] + ((data["high_ceilings"] < (data["price"] + (((data["doorman"] * 2.0) > -1.0).astype(float)))).astype(float))))))))))
    v["i40"] = np.tanh((-((((data["simplex"] + data["simplex"]) * ((data["furnished"] > (-((data["manager_level_low"] + (-(((data["building_id_mean_high"] + ((data["furnished"] < data["manager_level_low"]).astype(float)))/2.0))))))).astype(float))) * ((((data["manager_level_low"] + (-(((data["building_id_mean_high"] + ((data["building_id_mean_high"] < data["building_id_mean_medium"]).astype(float)))/2.0)))) * data["manager_level_low"]) < data["manager_level_low"]).astype(float))))))
    v["i41"] = np.tanh((((((np.tanh((-((data["outdoor_space"] * data["exclusive"])))) + data["central_a"])/2.0) < ((((((((data["outdoor"] - ((data["longitude"] * (np.tanh(data["outdoor"]) * 2.0)) / 2.0)) * data["site_super"]) - data["_photos"]) * np.tanh(data["longitude"])) * 2.0) + data["site_super"]) * 2.0) * 2.0)).astype(float)) / 2.0))
    v["i42"] = np.tanh((data["simplex"] * ((np.tanh(data["hardwood"]) > ((data["price"] + (data["walk"] + (data["hardwood"] - ((data["hardwood"] > (((data["manager_id_mean_medium"] + ((((data["bedrooms"] + ((data["price"] > (data["listing_id"] * 2.0)).astype(float)))/2.0) > ((np.tanh(data["hardwood"]) < data["walk"]).astype(float))).astype(float)))/2.0) / 2.0)).astype(float)))))/2.0)).astype(float))))
    v["i43"] = np.tanh((-((((data["price_per_bed"] + data["price_per_room"])/2.0) * (((((((-(((data["manager_level_low"] / 2.0) / 2.0))) + ((14.96606445312500000) * data["price_per_room"]))/2.0) * ((data["manager_level_high"] < (data["manager_level_medium"] / 2.0)).astype(float))) + (data["manager_level_low"] / 2.0)) > ((14.96606445312500000) * data["sublet"])).astype(float))))))
    v["i44"] = np.tanh((data["price_per_bath"] * np.tanh(((((((data["dining_room"] - data["new_construction"]) > data["price_per_bath"]).astype(float)) + data["reduced_fee"])/2.0) - ((((-(data["pre"])) * 2.0) * data["swimming_pool"]) - (((data["dining_room"] - data["new_construction"]) + (((data["building_id_mean_medium"] - data["price_per_bath"]) + (np.tanh(data["num_description_words"]) - data["renovated"]))/2.0))/2.0))))))
    v["i45"] = np.tanh(((data["price"] * 2.0) * (-((((((data["price_per_bed"] / 2.0) > (((data["terrace"] > (-(((data["no_fee"] * data["price_per_bed"]) - (data["view"] * data["live"]))))).astype(float)) - data["video_intercom"])).astype(float)) > (((data["no_fee"] - ((data["no_fee"] * data["private_outdoor_space"]) * data["private_outdoor_space"])) < data["manager_level_low"]).astype(float))).astype(float))))))
    v["i46"] = np.tanh((data["work"] * ((data["price"] < (data["simplex"] + ((data["new_construction"] < (-1.0 + ((data["building_id_mean_high"] < ((((((data["simplex"] > data["balcony"]).astype(float)) * ((data["building_id_mean_high"] < ((data["price"] < (data["multi"] * data["building_id_mean_high"])).astype(float))).astype(float))) / 2.0) + data["high_speed_internet"])/2.0)).astype(float)))).astype(float)))).astype(float))))
    v["i47"] = np.tanh(((((((((data["pre"] > data["elevator"]).astype(float)) * data["level"]) > data["balcony"]).astype(float)) * data["garden"]) < (data["cats_allowed"] + (-((((data["valet"] < (data["loft"] * (((data["longitude"] + data["valet"])/2.0) * data["level"]))).astype(float)) * 2.0))))).astype(float)))
    v["i48"] = np.tanh((-((((data["roofdeck"] + data["roofdeck"]) * ((data["latitude"] < (-(((((data["residents_garden"] > data["latitude"]).astype(float)) + data["residents_garden"])/2.0)))).astype(float))) * (((data["virtual_doorman"] > data["longitude"]).astype(float)) + (((data["indoor_pool"] > data["manager_level_medium"]).astype(float)) + ((((data["playroom"] > data["latitude"]).astype(float)) < ((data["price"] > data["longitude"]).astype(float))).astype(float))))))))
    v["i49"] = np.tanh(((((data["no_pets"] * (((((data["dishwasher"] > data["site_laundry"]).astype(float)) / 2.0) > data["manager_level_low"]).astype(float))) * 2.0) * 2.0) * (((((np.tanh(((data["dishwasher"] < data["no_pets"]).astype(float))) / 2.0) + data["site_laundry"]) * ((data["dishwasher"] > data["manager_level_low"]).astype(float))) > ((-(data["manager_level_low"])) * 2.0)).astype(float))))
    v["i50"] = np.tanh(np.tanh(np.tanh(((-((((data["valet_parking"] * data["laundry_in_building"]) * 2.0) * (((-((data["manager_level_high"] + (data["playroom"] - ((((data["num_description_words"] < data["laundry_in_building"]).astype(float)) + data["washer_"])/2.0))))) + (((data["manager_id_mean_high"] - data["washer"]) < data["lowrise"]).astype(float))) + ((data["num_description_words"] < data["speed_internet"]).astype(float)))))) / 2.0))))
    v["i51"] = np.tanh((data["common_terrace"] * (data["building_id_mean_medium"] - ((((np.tanh(((((data["building_id_mean_medium"] + (data["building_id_mean_medium"] - data["building_id_mean_high"]))/2.0) + ((data["simplex"] + data["common_outdoor_space"])/2.0))/2.0)) > (((data["price_per_bed"] / 2.0) < data["common_terrace"]).astype(float))).astype(float)) < (data["lounge_room"] * (data["bike_room"] - ((data["common_outdoor_space"] > data["common_terrace"]).astype(float))))).astype(float)))))
    v["i52"] = np.tanh(((data["new_construction"] * data["num_photos"]) * (((((data["num_photos"] * data["patio"]) < data["new_construction"]).astype(float)) < (((((data["num_photos"] * ((data["new_construction"] * data["num_photos"]) + data["price"])) < data["outdoor_space"]).astype(float)) < ((data["num_photos"] * ((data["outdoor_space"] * data["s_playroom"]) + data["price"])) + data["price"])).astype(float))).astype(float))))
    v["i53"] = np.tanh((data["hardwood"] * (-(((data["stainless_steel_appliances"] > ((((((((data["exposed_brick"] / 2.0) / 2.0) * (data["live_in_super"] * data["live_in_super"])) - data["live_in_super"]) * data["eat_in_kitchen"]) - data["live_in_super"]) * (-((((data["exposed_brick"] * data["building_id_mean_high"]) / 2.0) / 2.0)))) / 2.0)).astype(float))))))
    v["i54"] = np.tanh((-((((5.0) < (((data["bedrooms"] * 2.0) + (data["parking"] * ((((data["bedrooms"] * 2.0) * (-(data["parking_space"]))) + np.tanh((data["newly_renovated"] + (((data["no_fee"] + (data["bedrooms"] * 2.0))/2.0) + data["listing_id"])))) + (-(data["private_balcony"]))))) + (-(data["parking_space"])))).astype(float)))))
    v["i55"] = np.tanh(((data["newly_renovated"] * 2.0) * ((data["price_per_room"] > (data["swimming_pool"] - (-((data["building_id_mean_medium"] + (((data["dryer_in_building"] * data["price_per_room"]) > ((data["site_garage"] * data["dishwasher"]) + ((data["dryer_in_building"] * (data["furnished"] + data["gym"])) + (data["wheelchair_access"] + data["price_per_room"])))).astype(float))))))).astype(float))))
    v["i56"] = np.tanh((np.tanh(data["childrens_playroom"]) * ((-((((((data["building_id_mean_medium"] * data["doorman"]) > (data["building_id_mean_medium"] * (data["building_id_mean_medium"] * data["doorman"]))).astype(float)) < ((data["residents_garden"] < data["manager_level_low"]).astype(float))).astype(float)))) - ((data["building_id_mean_medium"] < (data["cats_allowed"] - ((data["doorman"] > data["manager_level_low"]).astype(float)))).astype(float)))))
    v["i57"] = np.tanh((((data["in_super"] * 2.0) * 2.0) * ((data["latitude"] < (((data["publicoutdoor"] - ((data["_dishwasher_"] > ((data["price_per_bath"] + ((data["in_super"] > ((data["price_per_bath"] + ((data["latitude"] > (data["in_super"] * 2.0)).astype(float)))/2.0)).astype(float)))/2.0)).astype(float))) - (data["in_super"] * (((data["price_per_room"] > (data["in_super"] * 2.0)).astype(float)) / 2.0))) / 2.0)).astype(float))))
    v["i58"] = np.tanh((data["manager_level_high"] * (data["_dryer"] * ((data["washer"] > (-((((((data["reduced_fee"] + (data["short_term_allowed"] * data["manager_level_high"])) > (-(data["exclusive"]))).astype(float)) > (data["bathrooms"] * (((data["longitude"] / 2.0) > (-(((data["washer"] + data["pool"]) * (-(data["washer"])))))).astype(float)))).astype(float))))).astype(float)))))
    v["i59"] = np.tanh((-((((data["fitness"] * ((data["fitness_center"] - data["roof"]) - data["new_construction"])) * ((data["fitness_center"] - ((data["no_fee"] + data["cats_allowed"])/2.0)) - (data["fitness_center"] * data["building_id_mean_high"]))) * ((data["new_construction"] < (-1.0 - data["building_id_mean_high"])).astype(float))))))
    v["i60"] = np.tanh((data["laundry"] * ((data["unit_washer"] > (data["latitude"] * (-(((data["wifi_access"] < (data["latitude"] * (-(((data["wifi_access"] < ((data["manager_id_mean_medium"] + ((((data["num_description_words"] + data["laundry_in_building"])/2.0) > (((data["manager_id_mean_medium"] < data["latitude"]).astype(float)) * ((data["manager_id_mean_medium"] + data["laundry"])/2.0))).astype(float)))/2.0)).astype(float)))))).astype(float)))))).astype(float))))
    v["i61"] = np.tanh((-((((data["price_per_room"] * ((((data["laundry_in_building"] < ((data["building_id_mean_medium"] + data["new_construction"])/2.0)).astype(float)) > ((data["price_per_bath"] < ((data["furnished"] + (data["laundry_in_building"] * (data["price_per_room"] / 2.0)))/2.0)).astype(float))).astype(float))) + (data["price_per_bath"] * ((data["laundry_in_building"] < ((data["new_construction"] + (data["price_per_bath"] - data["furnished"]))/2.0)).astype(float))))/2.0))))
    v["i62"] = np.tanh((((data["simplex"] * ((data["longitude"] > data["building"]).astype(float))) * ((data["wheelchair_ramp"] > (-(((data["longitude"] * ((((data["dishwasher"] > (-(data["post"]))).astype(float)) > (-(((((data["dogs_allowed"] > data["longitude"]).astype(float)) > (((data["manager_id_mean_high"] > data["building"]).astype(float)) * data["manager_id_mean_high"])).astype(float))))).astype(float))) / 2.0)))).astype(float))) * 2.0))
    v["i63"] = np.tanh(((data["site_garage"] - ((data["central_a"] < data["walk_in_closet"]).astype(float))) * (-((data["latitude"] - (-((((data["terrace"] * data["new_construction"]) < ((((data["terrace"] + data["price_per_room"]) - (data["roof_deck"] * data["terrace"])) + (((data["price_per_room"] * 2.0) * 2.0) + (data["new_construction"] + data["reduced_fee"])))/2.0)).astype(float)))))))))
    v["i64"] = np.tanh((-((((-(((data["actual_apt"] + ((np.tanh((data["courtyard"] * data["parking_space"])) + ((((data["listing_id"] < ((data["latitude"] < data["courtyard"]).astype(float))).astype(float)) + (data["latitude"] * data["parking_space"])) - data["listing_id"]))/2.0))/2.0))) > ((((data["latitude"] < ((data["sublet"] < data["latitude"]).astype(float))).astype(float)) + data["latitude"])/2.0)).astype(float)))))
    v["i65"] = np.tanh((-((data["price_per_room"] * ((data["deck"] * ((((data["lounge_room"] + ((data["manager_id_mean_high"] * 2.0) - data["granite_kitchen"])) / 2.0) * 2.0) * 2.0)) * (((data["price_per_room"] * data["deck"]) < (((data["manager_id"] * np.tanh(data["manager_id"])) < np.tanh(data["manager_id_mean_high"])).astype(float))).astype(float)))))))
    v["i66"] = np.tanh((data["parking_space"] * ((((-(((((((data["latitude"] / 2.0) > (-(data["hardwood"]))).astype(float)) < ((data["latitude"] * 2.0) / 2.0)).astype(float)) * (data["storage"] * ((((data["manager_level_low"] > data["latitude"]).astype(float)) < (data["manager_level_low"] + (data["manager_level_low"] + ((data["storage"] > data["no_fee"]).astype(float))))).astype(float)))))) * 2.0) * 2.0) * 2.0)))
    v["i67"] = np.tanh(((data["price_per_room"] > ((((data["_photos"] > (data["private_terrace"] * (data["furnished"] * (data["gym_in_building"] + data["price_per_room"])))).astype(float)) < (-((data["business_center"] * ((data["_pets_ok_"] < ((data["furnished"] * (data["_photos"] * data["building_id_mean_medium"])) * ((data["gym_in_building"] > np.tanh((data["_photos"] + data["price_per_room"]))).astype(float)))).astype(float)))))).astype(float))).astype(float)))
    v["i68"] = np.tanh(((np.tanh((data["price_per_room"] * np.tanh(((((data["short_term_allowed"] + (((((data["longitude"] < data["post"]).astype(float)) + ((data["longitude"] + data["post"])/2.0))/2.0) - data["price_per_room"]))/2.0) * (((data["bedrooms"] / 2.0) < (data["gym"] + data["manager_level_medium"])).astype(float))) + (data["brownstone"] + data["gym"]))))) * 2.0) * 2.0))
    v["i69"] = np.tanh(((((data["fitness_center"] < data["bedrooms"]).astype(float)) + (-(data["price_per_room"]))) * (-(((data["bedrooms"] < (-(((((((data["fitness_center"] / 2.0) / 2.0) / 2.0) > (-(data["price_per_room"]))).astype(float)) + ((data["price_per_room"] / 2.0) + ((data["common_outdoor_space"] * 2.0) + ((data["bedrooms"] < (data["bathrooms"] / 2.0)).astype(float)))))))).astype(float))))))
    v["i70"] = np.tanh((data["price_per_bath"] * (data["lounge_room"] * ((9.869604 + 9.869604) * (((data["new_construction"] * (data["price_per_bath"] + ((data["private_outdoor_space"] * (((data["new_construction"] + data["price_per_bath"]) * data["private_outdoor_space"]) * ((data["laundry_in_building"] < data["gym_in_building"]).astype(float)))) - data["in_superintendent"]))) < (data["virtual_doorman"] * data["laundry_in_building"])).astype(float))))))
    v["i71"] = np.tanh((data["latitude"] * np.tanh((np.tanh((((data["reduced_fee"] + ((((data["dining_room"] + data["dining_room"])/2.0) + data["latitude"])/2.0))/2.0) - np.tanh((data["latitude"] - ((((data["reduced_fee"] + ((data["doorman"] + data["building_id_mean_medium"]) * data["manager_id"]))/2.0) - data["latitude"]) * 2.0))))) * 2.0))))
    v["i72"] = np.tanh(((data["site_super"] * data["building_id_mean_high"]) * ((((((data["shares_ok"] + ((data["price_per_room"] < data["parking_space"]).astype(float))) / 2.0) + (data["short_term_allowed"] - (data["parking_space"] - data["wheelchair_access"])))/2.0) + (data["building_id_mean_high"] * (-(((data["parking_space"] > ((data["price_per_room"] + (data["shares_ok"] + ((data["short_term_allowed"] < data["wheelchair_access"]).astype(float))))/2.0)).astype(float))))))/2.0)))
    v["i73"] = np.tanh(((data["live_in_super"] * ((data["bathrooms"] - (data["roof"] * ((data["furnished"] * (data["num_photos"] / 2.0)) - data["short_term_allowed"]))) * (-(((((((data["time_doorman"] > (data["furnished"] * data["outdoor_areas"])).astype(float)) > (np.tanh(data["indoor_pool"]) * data["roof"])).astype(float)) > (data["light"] * data["time_doorman"])).astype(float)))))) * 2.0))
    v["i74"] = np.tanh((((((data["wheelchair_access"] + ((((((data["marble_bath"] + ((data["high_speed_internet"] > (-((data["manager_id"] * data["manager_level_medium"])))).astype(float)))/2.0) + data["manager_level_medium"])/2.0) > ((((data["wheelchair_access"] + (data["high_speed_internet"] * 2.0))/2.0) < data["manager_level_medium"]).astype(float))).astype(float)))/2.0) * 2.0) * 2.0) * (data["site_parking"] * (data["manager_level_medium"] * data["common_outdoor_space"]))))
    v["i75"] = np.tanh((data["private_balcony"] * ((data["manager_level_high"] < (((-(data["latitude"])) - (data["newly_renovated"] * ((-(((data["private_balcony"] < (-(data["manager_id_mean_high"]))).astype(float)))) + (-(data["cats_allowed"]))))) - (((data["manager_id_mean_high"] / 2.0) < data["manager_level_low"]).astype(float)))).astype(float))))
    v["i76"] = np.tanh((data["dishwasher"] * (((data["indoor_pool"] * 2.0) * 2.0) * ((((((data["longitude"] * 2.0) * 2.0) < data["backyard"]).astype(float)) + (-((((data["short_term_allowed"] - ((((data["longitude"] * data["indoor_pool"]) < (data["s_playroom"] - data["longitude"])).astype(float)) * data["dishwasher"])) < (np.tanh(data["s_playroom"]) - data["longitude"])).astype(float)))))/2.0))))
    v["i77"] = np.tanh((data["dryer_in_unit"] * (data["dryer_in_unit"] * (data["_pets_ok_"] * ((data["light"] + (data["_pets_ok_"] * (data["sublet"] + ((data["building_id_mean_medium"] < ((data["_pets_ok_"] < (data["cats_allowed"] * data["valet"])).astype(float))).astype(float))))) + ((data["building_id_mean_medium"] < (data["high_ceilings"] + ((data["building_id_mean_medium"] < (data["cats_allowed"] * data["light"])).astype(float)))).astype(float)))))))
    v["i78"] = np.tanh(((data["wifi_access"] * data["building_id_mean_medium"]) * (((data["reduced_fee"] - ((data["building_id"] < (data["no_fee"] - (data["building_id_mean_high"] * (data["building_id_mean_medium"] / 2.0)))).astype(float))) > (((((data["no_fee"] - data["common_outdoor_space"]) - data["common_outdoor_space"]) + data["manager_level_high"]) + (((data["subway"] * data["manager_level_high"]) * 2.0) * data["building_id_mean_high"]))/2.0)).astype(float))))
    v["i79"] = np.tanh(((((data["indoor_pool"] * (data["listing_id"] * 2.0)) + (-(((data["balcony"] > (((data["building_id_mean_medium"] + (((-(data["manager_level_low"])) + (((data["balcony"] * data["building_id_mean_medium"]) < 0.636620).astype(float)))/2.0))/2.0) / 2.0)).astype(float)))))/2.0) * (((data["bathrooms"] / 2.0) + ((data["balcony"] < data["manager_level_low"]).astype(float)))/2.0)))
    v["i80"] = np.tanh((-(((np.tanh(data["central_ac"]) + (data["no_fee"] * (-((data["new_construction"] * np.tanh((data["24"] * (data["doorman"] + ((data["outdoor_space"] + ((((data["24"] + data["terrace"])/2.0) - (data["no_fee"] - ((data["doorman"] + data["brownstone"])/2.0))) - data["dining_room"]))/2.0))))))))) / 2.0))))
    v["i81"] = np.tanh((data["manager_id_mean_high"] * (((((((data["common_roof_deck"] / 2.0) > data["longitude"]).astype(float)) + ((data["reduced_fee"] > (data["no_fee"] * data["price"])).astype(float))) + ((((data["no_fee"] - data["terrace"]) > data["reduced_fee"]).astype(float)) - ((data["price"] > (data["newly_renovated"] * 2.0)).astype(float)))) * data["price"]) / 2.0)))
    v["i82"] = np.tanh((data["bedrooms"] * ((((data["indoor_pool"] > (-(((data["common_roof_deck"] > (-(((data["site_parking"] > (-(data["_dishwasher_"]))).astype(float))))).astype(float))))).astype(float)) > (data["num_description_words"] * ((data["common_roof_deck"] > (-(((data["_dishwasher_"] > (-(((((data["fitness"] > (-(data["site_parking"]))).astype(float)) > ((data["site_parking"] < data["bedrooms"]).astype(float))).astype(float))))).astype(float))))).astype(float)))).astype(float))))
    v["i83"] = np.tanh((((-((data["wifi_access"] * (data["doorman"] * ((((data["wifi_access"] + (data["doorman"] * ((data["fireplace"] > ((((data["hardwood_floors"] > data["short_term_allowed"]).astype(float)) * data["short_term_allowed"]) * (data["manager_level_high"] * 2.0))).astype(float))))/2.0) > ((((data["hardwood_floors"] > data["manager_level_high"]).astype(float)) * data["reduced_fee"]) * data["furnished"])).astype(float)))))) * 2.0) * 2.0))
    v["i84"] = np.tanh(((-((data["price_per_bath"] * ((data["central_a"] > (data["listing_id"] * (data["balcony"] * ((data["work"] > (data["listing_id"] * (data["short_term_allowed"] * (((((data["work"] < (data["building_id_mean_medium"] * data["central_a"])).astype(float)) > data["price_per_bath"]).astype(float)) / 2.0)))).astype(float))))).astype(float))))) * 2.0))
    v["i85"] = np.tanh((data["latitude"] * ((((-(data["s_kitchen_"])) - data["price"]) > ((data["price"] > ((((-(data["num_photos"])) > ((data["price"] > (data["furnished"] - (data["price"] * data["price_per_bed"]))).astype(float))).astype(float)) - (((data["price_per_bed"] * data["high_ceilings"]) - data["fitness_center"]) - data["furnished"]))).astype(float))).astype(float))))
    v["i86"] = np.tanh((((data["washer"] * (((((((data["_dishwasher_"] + data["latitude"]) + ((data["washer"] > (((data["_dishwasher_"] + data["latitude"]) + ((data["_dishwasher_"] > ((data["price_per_bath"] < (data["washer"] / 2.0)).astype(float))).astype(float)))/2.0)).astype(float)))/2.0) > (-((data["washer"] / 2.0)))).astype(float)) > (((data["price_per_bath"] * 2.0) < data["washer"]).astype(float))).astype(float))) * 2.0) * 2.0))
    v["i87"] = np.tanh(((data["business_center"] * (data["manager_id_mean_medium"] - data["furnished"])) * (((((data["manager_id_mean_medium"] - data["site_garage"]) > (((data["price_per_room"] * (data["outdoor"] - data["washer"])) > data["price_per_room"]).astype(float))).astype(float)) - (((data["manager_id_mean_medium"] * (data["price_per_room"] - data["washer"])) > (data["price_per_room"] - data["wheelchair_ramp"])).astype(float))) + data["washer"])))
    v["i88"] = np.tanh(((data["longitude"] + data["residents_garden"]) * (-((((data["childrens_playroom"] + (data["wheelchair_ramp"] * data["manager_level_medium"])) < (data["live"] * ((data["residents_garden"] < ((data["manager_level_medium"] - ((data["manager_level_high"] > ((data["fireplace"] < (data["childrens_playroom"] - ((data["exclusive"] > (data["no_pets"] - data["longitude"])).astype(float)))).astype(float))).astype(float))) / 2.0)).astype(float)))).astype(float))))))
    v["i89"] = np.tanh((((data["valet_parking"] * ((data["price_per_bed"] > ((((data["shares_ok"] > ((data["price_per_room"] * data["manager_id_mean_high"]) - (data["price_per_bed"] * ((np.tanh(data["shares_ok"]) > data["manager_id_mean_medium"]).astype(float))))).astype(float)) + (((data["bathrooms"] * data["garage"]) + ((data["manager_level_low"] > ((data["valet_parking"] > data["price_per_bed"]).astype(float))).astype(float)))/2.0))/2.0)).astype(float))) * 2.0) * 2.0))
    v["i90"] = np.tanh((np.tanh((((data["price_per_room"] * 3.141593) < (0.318310 / 2.0)).astype(float))) * (data["lowrise"] * (-((((((data["manager_level_medium"] / 2.0) * (data["price_per_room"] * data["lowrise"])) < (0.318310 / 2.0)).astype(float)) * (((data["manager_level_medium"] < (-((data["price_per_room"] * data["price_per_room"])))).astype(float)) * data["loft"])))))))
    v["i91"] = np.tanh((data["reduced_fee"] * ((data["building_id"] * ((((data["common_roof_deck"] + (data["longitude"] * (((data["reduced_fee"] + (data["pets_on_approval"] * data["building_id"]))/2.0) * ((data["manager_level_high"] - (data["outdoor_space"] - data["longitude"])) - ((data["pets_on_approval"] + data["pets_on_approval"])/2.0)))))/2.0) + (data["pet_friendly"] / 2.0))/2.0)) / 2.0)))
    v["i92"] = np.tanh((((-(data["terrace"])) - data["s_kitchen_"]) * np.tanh((data["pool"] * (((data["longitude"] < data["parking"]).astype(float)) + ((data["marble_bath"] > (((data["marble_bath"] - data["manager_level_low"]) - ((data["longitude"] < data["parking"]).astype(float))) * ((data["multi"] > (((data["terrace"] < data["manager_level_low"]).astype(float)) - data["manager_id_mean_high"])).astype(float)))).astype(float)))))))
    v["i93"] = np.tanh((data["valet_parking"] * np.tanh(((((data["price_per_room"] * 2.0) + data["building_id_mean_medium"])/2.0) + (data["dining_room"] + (((((data["price_per_room"] * 2.0) * 2.0) * 2.0) + ((data["manager_id_mean_high"] - data["num_photos"]) - data["num_photos"])) - ((data["private_outdoor_space"] + data["reduced_fee"])/2.0)))))))
    v["i94"] = np.tanh(((data["post_war"] > (data["longitude"] * (((data["price"] + data["post_war"]) + (((((data["price"] > data["post_war"]).astype(float)) - data["price"]) * 2.0) * ((data["eat_in_kitchen"] > data["longitude"]).astype(float)))) * ((data["fireplace"] > (data["price"] + (data["post_war"] * data["price_per_room"]))).astype(float))))).astype(float)))
    v["i95"] = np.tanh(((data["dryer_in_building"] * ((((data["building_id"] + data["garden"]) > data["no_fee"]).astype(float)) - ((data["business_center"] < (data["hi_rise"] + (data["cats_allowed"] * (data["new_construction"] * ((((data["no_fee"] + data["garden"])/2.0) * data["building_id_mean_medium"]) + ((data["hi_rise"] + data["no_fee"])/2.0)))))).astype(float)))) * 2.0))
    v["i96"] = np.tanh((data["parking_space"] * (((data["parking_space"] * ((data["high_speed_internet"] > data["reduced_fee"]).astype(float))) > (-(((((data["simplex"] > (-(((data["price_per_room"] + (data["common_outdoor_space"] * (data["simplex"] * ((data["in_super"] > data["parking_space"]).astype(float)))))/2.0)))).astype(float)) + ((data["luxury_building"] > (((data["air_conditioning"] + data["roofdeck"])/2.0) * data["price_per_room"])).astype(float)))/2.0)))).astype(float))))
    v["i97"] = np.tanh(((((data["price"] * 2.0) + ((data["manager_id"] < (((data["manager_id"] + data["price"])/2.0) - (data["building_id_mean_high"] + data["manager_id_mean_medium"]))).astype(float))) * ((data["parking_space"] > ((data["price"] + data["highrise"])/2.0)).astype(float))) * ((data["building_id_mean_high"] > data["parking_space"]).astype(float))))
    v["i98"] = np.tanh(((data["bathrooms"] + ((data["simplex"] + data["bedrooms"])/2.0)) * (((data["shares_ok"] > ((data["latitude"] + ((((data["duplex"] + (data["manager_id"] + data["shares_ok"]))/2.0) > ((((((data["no_pets"] + data["reduced_fee"])/2.0) < data["reduced_fee"]).astype(float)) < ((data["manager_level_medium"] < (data["manager_id"] + data["simplex"])).astype(float))).astype(float))).astype(float)))/2.0)).astype(float)) * data["simplex"])))
    v["i99"] = np.tanh(np.tanh((-(((data["walk"] * 2.0) * np.tanh(np.tanh((((((((data["balcony"] + ((data["manager_id_mean_medium"] + ((data["building_id_mean_high"] + data["site_laundry"])/2.0)) - ((data["building_id_mean_high"] < data["doorman"]).astype(float)))) * 2.0) * 2.0) + data["bedrooms"]) + data["manager_id_mean_medium"]) * 2.0) + data["building_id_mean_high"]))))))))
    return Outputs(v.sum(axis=1))

def GPTwo(data):
    v = pd.DataFrame()
    v["i0"] = np.tanh((data["manager_id_mean_high"] - (((((((-((data["building_id_mean_high"] + (((data["site_parking"] > (data["price_per_bed"] + (-((data["concierge"] / 2.0))))).astype(float)) - (data["manager_id_mean_high"] - data["manager_level_high"]))))) / 2.0) * 2.0) / 2.0) * 2.0) + (9.38112831115722656))/2.0)))
    v["i1"] = np.tanh((((((-(((data["manager_level_high"] < ((((((data["building_id_mean_high"] + data["price_per_bed"])/2.0) * 2.0) > (data["sublet"] + data["building_id_mean_high"])).astype(float)) + (((data["furnished"] < (data["price_per_bed"] + ((3.141593 > (((data["building_id_mean_high"] + data["price_per_bed"])/2.0) * 2.0)).astype(float)))).astype(float)) - np.tanh(data["building_id_mean_high"])))).astype(float)))) * 2.0) * 2.0) * 2.0) * 2.0))
    v["i2"] = np.tanh(((((data["short_term_allowed"] + data["_photos"]) - (data["price_per_room"] * 2.0)) - ((data["manager_level_low"] > data["building_id_mean_high"]).astype(float))) - ((((data["price"] > (((data["short_term_allowed"] + data["renovated"]) * data["manager_level_low"]) + (data["health_club"] + data["short_term_allowed"]))).astype(float)) - ((((data["price"] + data["short_term_allowed"])/2.0) < data["renovated"]).astype(float))) * 2.0)))
    v["i3"] = np.tanh((-((data["price_per_room"] - ((-(((data["price"] > ((data["dryer_in_unit"] * (-(data["no_fee"]))) * 2.0)).astype(float)))) - (data["price_per_bed"] - (np.tanh((np.tanh(((data["building_id_mean_high"] + data["furnished"])/2.0)) * 2.0)) * ((((np.tanh(((data["building_id_mean_high"] + data["furnished"])/2.0)) + data["manager_level_high"])/2.0) < data["furnished"]).astype(float)))))))))
    v["i4"] = np.tanh((-((data["price"] + (data["price"] + ((((data["virtual_doorman"] < ((data["longitude"] + (data["latitude"] + (data["longitude"] * 2.0))) * 2.0)).astype(float)) + ((((data["garage"] + ((data["latitude"] < data["outdoor"]).astype(float)))/2.0) + (np.tanh(((data["garage"] + (data["subway"] - data["manager_id_mean_medium"]))/2.0)) * 2.0))/2.0))/2.0))))))
    v["i5"] = np.tanh(((-(((data["site_garage"] > (data["price"] - data["simplex"])).astype(float)))) - (((data["price"] > (data["site_garage"] - ((data["price"] > (data["site_garage"] - (-(data["live_in_super"])))).astype(float)))).astype(float)) - (((-(((1.570796 > data["in_superintendent"]).astype(float)))) < ((data["num_photos"] + (-(((1.570796 + data["live_in_super"])/2.0))))/2.0)).astype(float)))))
    v["i6"] = np.tanh((-(((np.tanh(((data["washer_in_unit"] + ((((data["price_per_bath"] + (data["exclusive"] * data["no_fee"]))/2.0) + (data["exclusive"] * data["laundry_in_unit"]))/2.0))/2.0)) + (data["price_per_bath"] + ((data["price_per_room"] > ((data["washer_in_unit"] + (((data["no_fee"] / 2.0) > ((data["valet"] < data["price_per_bath"]).astype(float))).astype(float)))/2.0)).astype(float))))/2.0))))
    v["i7"] = np.tanh((((np.tanh((np.tanh(((-(data["manager_id_mean_medium"])) + ((-(data["manager_id_mean_high"])) * 2.0))) * data["building_id_mean_high"])) + (((((data["manager_id_mean_high"] * (data["manager_id_mean_high"] * ((data["walk"] + data["price_per_bed"]) + (data["fitness"] * data["manager_id_mean_high"])))) + data["manager_id_mean_high"])/2.0) + ((data["common_parking"] > data["price_per_bed"]).astype(float)))/2.0))/2.0) / 2.0))
    v["i8"] = np.tanh((-(((((((3.141593 - data["listing_id"]) < ((((3.141593 > data["longitude"]).astype(float)) + data["listing_id"])/2.0)).astype(float)) * 2.0) * 2.0) + (np.tanh(data["longitude"]) * (-((((((data["doorman"] - data["building_id_mean_medium"]) * 2.0) - data["laundry_in_building"]) - ((data["hi_rise"] < (data["longitude"] / 2.0)).astype(float))) * 2.0))))))))
    v["i9"] = np.tanh((((-((((((((((data["price_per_bed"] > data["subway"]).astype(float)) + data["price_per_bed"]) > data["latitude"]).astype(float)) > data["bedrooms"]).astype(float)) + ((data["subway"] > data["latitude"]).astype(float)))/2.0))) + ((data["reduced_fee"] > (data["subway"] + ((data["price"] + (((data["price_per_bed"] > data["lounge"]).astype(float)) + data["subway"]))/2.0))).astype(float)))/2.0))
    v["i10"] = np.tanh((((data["garage"] > data["price"]).astype(float)) * ((data["bedrooms"] - (data["balcony"] + (data["latitude"] + data["garage"]))) + ((data["outdoor_space"] > (((data["bedrooms"] + (data["price_per_bed"] + data["wheelchair_access"])) + 9.869604) * ((data["price_per_bed"] + data["common_terrace"])/2.0))).astype(float)))))
    v["i11"] = np.tanh((-((-(((((data["common_outdoor_space"] > ((data["sublet"] + (-(data["garage"])))/2.0)).astype(float)) + ((data["price_per_bed"] > ((((((data["no_fee"] > data["garage"]).astype(float)) < data["price_per_bed"]).astype(float)) + (data["virtual_doorman"] + (((-(data["bathrooms"])) < ((data["price_per_bed"] * 2.0) + data["price_per_bed"])).astype(float))))/2.0)).astype(float)))/2.0))))))
    v["i12"] = np.tanh(((data["dishwasher"] * data["price_per_room"]) + (((data["latitude"] * ((data["laundry_in_building"] - data["latitude"]) + (((-(data["latitude"])) + ((-(((data["price_per_room"] < data["latitude"]).astype(float)))) + (((data["latitude"] < data["_photos"]).astype(float)) * 2.0))) * 2.0))) + (-(data["site_garage"]))) + (-(data["site_garage"])))))
    v["i13"] = np.tanh(((data["publicoutdoor"] + data["concierge"]) * (((data["loft"] + (data["no_fee"] + ((data["publicoutdoor"] < data["price_per_room"]).astype(float))))/2.0) - (((((data["no_fee"] + (-(((data["price"] > data["publicoutdoor"]).astype(float))))) > ((data["loft"] < data["no_fee"]).astype(float))).astype(float)) * 2.0) * 2.0))))
    v["i14"] = np.tanh(((data["common_parking"] - (((data["bathrooms"] < (-(((((data["sauna"] * data["highrise"]) + data["garage"]) + (data["playroom"] + ((data["concierge"] < data["longitude"]).astype(float))))/2.0)))).astype(float)) * 2.0)) - ((data["longitude"] + ((data["green_building"] - data["longitude"]) * (((data["longitude"] * 2.0) < data["playroom"]).astype(float)))) * 2.0)))
    v["i15"] = np.tanh(((((data["building_id_mean_high"] + np.tanh((data["war"] * 2.0)))/2.0) * np.tanh(data["price_per_bed"])) - np.tanh((data["parking"] * (data["terrace"] - ((data["roof_deck"] + ((data["terrace"] * (data["pre"] * data["building_id_mean_high"])) - (((data["building_id_mean_high"] + data["24"])/2.0) * data["laundry_in_unit"]))) - data["laundry_in_unit"]))))))
    v["i16"] = np.tanh((data["longitude"] - (((data["price_per_room"] * 2.0) + ((((data["walk"] < ((data["price"] + (-((data["green_building"] * ((((data["price_per_room"] > (data["green_building"] * ((data["price"] < data["green_building"]).astype(float)))).astype(float)) < data["longitude"]).astype(float))))))/2.0)).astype(float)) > ((data["walk"] > ((data["longitude"] * 2.0) * 2.0)).astype(float))).astype(float)))/2.0)))
    v["i17"] = np.tanh((((data["num_photos"] * ((data["num_photos"] * ((data["walk"] + ((data["price_per_room"] * ((((data["manager_id_mean_high"] + data["manager_level_low"])/2.0) + (data["manager_id_mean_medium"] - ((data["walk"] > data["manager_id_mean_high"]).astype(float))))/2.0)) * ((data["manager_id_mean_high"] + (data["reduced_fee"] * 2.0))/2.0)))/2.0)) * 2.0)) + ((((-1.0 + data["manager_level_low"])/2.0) > data["price_per_bed"]).astype(float)))/2.0))
    v["i18"] = np.tanh((((np.tanh(np.tanh(data["price"])) > (-((((data["garage"] + data["light"]) + ((data["light"] * (((data["price_per_bed"] > data["light"]).astype(float)) - data["subway"])) + (((data["manager_level_medium"] - data["price_per_bed"]) > (3.141593 * ((data["air_conditioning"] > data["live"]).astype(float)))).astype(float))))/2.0)))).astype(float)) / 2.0))
    v["i19"] = np.tanh((((-((data["price_per_bath"] * np.tanh(np.tanh(((data["num_description_words"] * (((data["price_per_bath"] + data["garage"]) > (data["num_description_words"] + data["num_description_words"])).astype(float))) - ((-(((data["price"] < (data["garage"] - (-(((data["light"] < data["price_per_bath"]).astype(float)))))).astype(float)))) - (-(data["concierge"]))))))))) * 2.0) * 2.0))
    v["i20"] = np.tanh((data["price_per_room"] * ((data["high_speed_internet"] + ((((data["latitude"] - ((data["wheelchair_access"] > data["price_per_room"]).astype(float))) + ((((data["video_intercom"] < (data["full_service_garage"] + (data["latitude"] * (data["latitude"] / 2.0)))).astype(float)) + ((data["longitude"] > data["latitude"]).astype(float))) - ((data["high_ceilings"] > data["price_per_room"]).astype(float)))) * 2.0) * 2.0))/2.0)))
    v["i21"] = np.tanh(((data["gym"] * 2.0) * ((-(((data["building_id_mean_medium"] > (data["cats_allowed"] * ((data["price"] > data["laundry"]).astype(float)))).astype(float)))) + (((data["cats_allowed"] < ((-((data["manager_id_mean_high"] * np.tanh((((-((data["listing_id"] * 2.0))) < ((data["laundry"] > data["price"]).astype(float))).astype(float)))))) * 2.0)).astype(float)) * 2.0))))
    v["i22"] = np.tanh((-((((((data["midrise"] * data["private_backyard"]) < (((((data["sublet"] * data["price_per_bed"]) < (data["common"] + ((data["speed_internet"] + data["wifi_access"]) + (data["price_per_bed"] / 2.0)))).astype(float)) + data["speed_internet"]) + ((data["gym"] > ((((data["price_per_bed"] / 2.0) + data["price_per_bath"])/2.0) / 2.0)).astype(float)))).astype(float)) * 2.0) * 2.0))))
    v["i23"] = np.tanh((data["high_speed_internet"] * ((data["latitude"] * 2.0) * (np.tanh((((data["subway"] + data["num_description_words"]) + np.tanh((data["dogs_allowed"] * ((data["subway"] + (data["num_description_words"] - data["high_speed_internet"]))/2.0)))) + (data["latitude"] * 2.0))) - (data["private_outdoor_space"] + (data["laundry_room"] + data["laundry_in_building"]))))))
    v["i24"] = np.tanh((-((data["price_per_bed"] * (((((data["hardwood"] > data["price"]).astype(float)) + data["price_per_bed"]) + ((data["site_parking_lot"] < (-(data["manager_level_high"]))).astype(float))) + ((data["price_per_room"] < (-((data["sublet"] + ((data["price"] < (data["manager_level_high"] - ((((data["price_per_room"] < data["price_per_bed"]).astype(float)) - data["dogs_allowed"]) / 2.0))).astype(float)))))).astype(float)))))))
    v["i25"] = np.tanh((((data["in_super"] * 2.0) * 2.0) * ((data["bathrooms"] > (data["num_photos"] * (((((data["price_per_room"] + ((data["in_super"] * 2.0) * 2.0))/2.0) * (((data["price_per_room"] * 2.0) > data["price_per_room"]).astype(float))) > (data["num_photos"] * ((data["price_per_room"] < (data["manager_id"] * (((data["manager_level_low"] * 2.0) > data["bathrooms"]).astype(float)))).astype(float)))).astype(float)))).astype(float))))
    v["i26"] = np.tanh(((-((data["central_ac"] + (((data["simplex"] * (((data["building_id_mean_high"] < data["bathrooms"]).astype(float)) * 2.0)) + (((((data["building_id_mean_high"] < data["bathrooms"]).astype(float)) * 2.0) * ((data["building_id_mean_medium"] < data["pre"]).astype(float))) * ((((data["building_id_mean_high"] < data["building_id_mean_medium"]).astype(float)) < (data["manager_level_high"] * data["washer_in_unit"])).astype(float))))/2.0)))) - data["longitude"]))
    v["i27"] = np.tanh((data["price_per_room"] * (-((((data["manager_level_low"] < ((data["manager_level_low"] + (data["price_per_room"] + (data["building_id_mean_medium"] * (data["manager_level_low"] + (data["price_per_room"] * (((((data["newly_renovated"] + data["price_per_room"]) > (data["latitude"] * 2.0)).astype(float)) > (data["manager_level_low"] + data["gym"])).astype(float))))))) * 2.0)).astype(float)) * 2.0)))))
    v["i28"] = np.tanh(((((data["latitude"] + (((((data["latitude"] + (data["storage"] - ((data["site_laundry"] + ((data["building_id_mean_high"] < (-(data["latitude"]))).astype(float)))/2.0)))/2.0) - (((data["bike_room"] < data["price"]).astype(float)) - data["business_center"])) - data["price"]) - data["price"]))/2.0) * ((data["latitude"] < (data["business_center"] * 2.0)).astype(float))) * 2.0))
    v["i29"] = np.tanh((-((data["longitude"] - (-(((data["site_garage"] / 2.0) - ((((data["listing_id"] - data["lounge_room"]) - data["no_fee"]) - (data["swimming_pool"] / 2.0)) * (((data["listing_id"] - data["private_backyard"]) * (data["private_backyard"] * (data["listing_id"] - data["simplex"]))) * (data["listing_id"] - data["bedrooms"]))))))))))
    v["i30"] = np.tanh((-((data["manager_id_mean_high"] * (data["listing_id"] * (data["microwave"] * ((-(data["renovated"])) - (((((data["renovated"] + ((((((data["manager_id_mean_high"] * data["listing_id"]) + data["manager_id_mean_high"]) > (data["no_fee"] + data["listing_id"])).astype(float)) * data["manager_id_mean_high"]) / 2.0))/2.0) / 2.0) * data["manager_id_mean_high"]) / 2.0))))))))
    v["i31"] = np.tanh(((data["cats_allowed"] * (((((data["price_per_bed"] > (-(data["level"]))).astype(float)) > ((-(data["live"])) - ((data["granite_kitchen"] - ((-(data["wheelchair_access"])) - (data["simplex"] * ((data["pets_on_approval"] + ((data["building_id"] + data["bathrooms"])/2.0))/2.0)))) - ((-(data["air_conditioning"])) - data["parking"])))).astype(float)) / 2.0)) / 2.0))
    v["i32"] = np.tanh(((-((data["dryer_in_unit"] * np.tanh((data["furnished"] - (data["bedrooms"] * ((data["bedrooms"] * (data["bedrooms"] * (data["furnished"] + ((data["furnished"] - data["dishwasher"]) + ((data["washer"] + (((data["bedrooms"] * data["no_fee"]) + data["no_fee"])/2.0))/2.0))))) + data["no_fee"]))))))) * 2.0))
    v["i33"] = np.tanh((data["washer_"] * (data["dryer"] + ((-((data["private_outdoor_space"] - data["central_ac"]))) - (data["dishwasher"] + (((((data["latitude"] < (data["dryer"] * data["short_term_allowed"])).astype(float)) - data["live_in_super"]) - (data["building_id_mean_medium"] * data["building_id_mean_high"])) + ((((-(data["building_id_mean_high"])) < data["private_outdoor_space"]).astype(float)) - data["manager_id"])))))))
    v["i34"] = np.tanh((data["longitude"] * (data["cats_allowed"] * (((data["swimming_pool"] + (data["cats_allowed"] * ((((data["price_per_bath"] < (data["swimming_pool"] / 2.0)).astype(float)) + (data["building_id_mean_high"] * ((data["manager_level_medium"] < (data["manager_level_medium"] / 2.0)).astype(float)))) * 2.0))) + (data["cats_allowed"] * ((data["manager_level_medium"] < data["swimming_pool"]).astype(float)))) * 2.0))))
    v["i35"] = np.tanh((((data["longitude"] * (((((data["building_id"] - data["latitude"]) < data["hardwood_floors"]).astype(float)) + data["actual_apt"]) - ((((data["latitude"] < (((data["latitude"] < (data["longitude"] - data["latitude"])).astype(float)) * data["latitude"])).astype(float)) + ((data["granite_kitchen"] < (data["longitude"] - data["latitude"])).astype(float))) + data["longitude"]))) * 2.0) * 2.0))
    v["i36"] = np.tanh(((-(((data["microwave"] > ((((data["building_id_mean_high"] < data["s_kitchen_"]).astype(float)) + (-((((data["bedrooms"] + data["parking_space"])/2.0) * ((data["s_kitchen_"] + ((data["short_term_allowed"] > (data["fitness"] * (data["bedrooms"] - 1.570796))).astype(float))) + ((data["outdoor_areas"] > (data["short_term_allowed"] - data["video_intercom"])).astype(float)))))))/2.0)).astype(float)))) * 2.0))
    v["i37"] = np.tanh((data["laundry_in_building"] * (((data["lounge_room"] * ((1.85216593742370605) + ((data["health_club"] + ((data["laundry_in_building"] - data["fitness_center"]) - data["fitness_center"])) - (data["eat_in_kitchen"] * 2.0)))) * ((data["bedrooms"] - ((data["no_fee"] < data["bedrooms"]).astype(float))) - ((((1.85216593742370605) - data["bedrooms"]) < data["laundry_in_building"]).astype(float)))) / 2.0)))
    v["i38"] = np.tanh(((data["common_terrace"] + ((data["sauna"] > (-((data["health_club"] + ((data["longitude"] + (data["elevator"] * ((((((data["manager_level_low"] > data["health_club"]).astype(float)) - (data["manager_level_low"] * ((data["health_club"] > data["longitude"]).astype(float)))) * data["site_garage"]) + data["lounge"])/2.0)))/2.0))))).astype(float)))/2.0))
    v["i39"] = np.tanh((-(((((data["bathrooms"] < (-(((((data["washer"] * ((data["manager_id_mean_medium"] < data["reduced_fee"]).astype(float))) < ((data["laundry_in_building"] * data["outdoor_areas"]) - data["bathrooms"])).astype(float)) + ((((((2.0) < (data["laundry_in_building"] * data["laundry_in_building"])).astype(float)) * data["manager_id_mean_medium"]) + ((-((2.0))) / 2.0))/2.0))))).astype(float)) * 2.0) * 2.0))))
    v["i40"] = np.tanh((((((data["price_per_bed"] + (((data["marble_bath"] > ((data["price"] - data["deck"]) - data["marble_bath"])).astype(float)) - data["residents_lounge"]))/2.0) * ((-(((data["bedrooms"] > ((((((data["_dryer"] < data["bedrooms"]).astype(float)) - data["manager_level_high"]) - data["residents_lounge"]) < data["building_id_mean_high"]).astype(float))).astype(float)))) * 2.0)) * 2.0) * 2.0))
    v["i41"] = np.tanh(((data["residents_lounge"] + ((data["bedrooms"] * data["price_per_bath"]) / 2.0)) - (data["garage"] * ((((data["marble_bath"] > (-(((data["price"] * (((data["renovated"] * data["residents_lounge"]) > data["latitude"]).astype(float))) - (((data["manager_level_medium"] * data["residents_lounge"]) > (data["price"] * data["renovated"])).astype(float)))))).astype(float)) * 2.0) * 2.0))))
    v["i42"] = np.tanh((data["dryer_in_unit"] * ((data["longitude"] > (data["dryer_in_unit"] * (((((data["price_per_bath"] > ((data["longitude"] < data["shares_ok"]).astype(float))).astype(float)) / 2.0) > (-(((data["shares_ok"] > (((data["longitude"] < (data["latitude"] * 2.0)).astype(float)) - (data["manager_level_high"] - (((np.tanh(data["longitude"]) < (data["latitude"] * 2.0)).astype(float)) * 2.0)))).astype(float))))).astype(float)))).astype(float))))
    v["i43"] = np.tanh((-((data["roof"] * (((data["manager_level_high"] - ((np.tanh(data["building_id_mean_high"]) < (((data["manager_level_high"] * (0.0 - data["building_id"])) < (data["manager_id_mean_high"] - ((0.0 < data["building_id_mean_high"]).astype(float)))).astype(float))).astype(float))) < (((data["building_id"] < data["building_id_mean_high"]).astype(float)) + np.tanh((0.0 - data["building_id_mean_high"])))).astype(float))))))
    v["i44"] = np.tanh((data["deck"] * ((data["manager_level_medium"] < (((data["bathrooms"] - ((data["subway"] - ((data["building_id_mean_medium"] + ((((data["building_id_mean_medium"] + data["subway"])/2.0) < ((((data["exclusive"] * data["building_id_mean_medium"]) * 2.0) < data["deck"]).astype(float))).astype(float)))/2.0)) * ((data["high_speed_internet"] < data["building_id"]).astype(float)))) * 2.0) * 2.0)).astype(float))))
    v["i45"] = np.tanh((((((-(((data["price_per_bath"] + ((data["longitude"] > data["latitude"]).astype(float))) + ((data["common_parking"] > ((data["latitude"] + data["price_per_bath"])/2.0)).astype(float))))) > ((data["latitude"] > ((data["common_parking"] + (-(data["price_per_bath"]))) * 2.0)).astype(float))).astype(float)) * 2.0) * ((data["common_parking"] + (-(data["price_per_bath"]))) * 2.0)))
    v["i46"] = np.tanh(((data["lounge"] * ((((data["price_per_room"] > data["hi_rise"]).astype(float)) > (((data["building_id_mean_medium"] < data["price_per_room"]).astype(float)) * ((data["no_fee"] + data["price_per_room"])/2.0))).astype(float))) * (((((data["longitude"] > (((((data["price_per_room"] < ((data["building_id_mean_medium"] < data["price_per_room"]).astype(float))).astype(float)) + (-(data["listing_id"])))/2.0) * data["basement_storage"])).astype(float)) * 2.0) * 2.0) * 2.0)))
    v["i47"] = np.tanh(np.tanh((np.tanh((((((data["common_roof_deck"] - (-(data["pet_friendly"]))) + data["price_per_room"]) > (-((data["common_roof_deck"] - (-(data["private_terrace"])))))).astype(float)) * ((data["virtual_doorman"] + ((data["manager_level_low"] + (((data["roof_deck"] > (-(data["common_roof_deck"]))).astype(float)) - ((data["private_terrace"] > data["doorman"]).astype(float))))/2.0))/2.0))) - data["pet_friendly"])))
    v["i48"] = np.tanh((((-(((data["washer_in_unit"] < np.tanh(data["price"])).astype(float)))) * ((data["longitude"] > (data["price"] * (data["actual_apt"] - data["price"]))).astype(float))) * ((data["longitude"] > ((((data["building_id_mean_medium"] < data["price"]).astype(float)) + (-(data["fitness_center"]))) * data["common_terrace"])).astype(float))))
    v["i49"] = np.tanh((data["price"] * (((((((data["shares_ok"] > data["longitude"]).astype(float)) - ((data["manager_id_mean_medium"] < data["manager_level_medium"]).astype(float))) - ((data["manager_id_mean_medium"] < data["manager_level_medium"]).astype(float))) + ((((data["shares_ok"] > data["longitude"]).astype(float)) - data["dishwasher"]) - ((((data["manager_id_mean_medium"] > data["longitude"]).astype(float)) < data["manager_level_medium"]).astype(float))))/2.0) - ((data["manager_id_mean_medium"] < data["doorman"]).astype(float)))))
    v["i50"] = np.tanh(((data["actual_apt"] * 2.0) * ((data["site_super"] > (-(((data["listing_id"] + (((((data["price_per_bed"] < ((data["exposed_brick"] * 2.0) * (data["fireplace"] / 2.0))).astype(float)) < (data["manager_id_mean_high"] * 2.0)).astype(float)) - (data["manager_id_mean_high"] * 2.0))) + data["price_per_room"])))).astype(float))))
    v["i51"] = np.tanh(np.tanh(((((data["latitude"] + (data["publicoutdoor"] * 2.0)) * 2.0) + ((data["valet"] > data["longitude"]).astype(float))) * (-(((np.tanh(((((((((data["longitude"] + np.tanh(data["air_conditioning"]))/2.0) > data["wheelchair_ramp"]).astype(float)) + data["longitude"])/2.0) > data["latitude"]).astype(float))) < (-(((data["latitude"] + data["site_parking"])/2.0)))).astype(float)))))))
    v["i52"] = np.tanh((np.tanh(((((-(data["price_per_bath"])) < (((data["building_id_mean_medium"] + ((data["latitude"] + data["washer_"])/2.0))/2.0) * data["latitude"])).astype(float)) * (((data["num_photos"] / 2.0) + (-((((((data["latitude"] * 2.0) > data["price_per_bath"]).astype(float)) > ((data["building_id_mean_medium"] + ((((data["latitude"] > data["price_per_bath"]).astype(float)) + data["latitude"])/2.0))/2.0)).astype(float)))))/2.0))) / 2.0))
    v["i53"] = np.tanh((data["deck"] * (-(((data["manager_level_medium"] > ((((data["manager_level_medium"] > ((data["listing_id"] > (((((((data["walk"] > data["parking_space"]).astype(float)) > ((data["_photos"] + data["listing_id"]) * 2.0)).astype(float)) - data["roof"]) - data["parking_space"]) - data["parking_space"])).astype(float))).astype(float)) - (data["manager_level_medium"] * data["multi"])) - data["multi"])).astype(float))))))
    v["i54"] = np.tanh(((5.0) * (data["longitude"] * ((data["patio"] > (-((data["latitude"] - (-((data["longitude"] - (((data["manager_id_mean_high"] > ((data["manager_id_mean_high"] > data["longitude"]).astype(float))).astype(float)) - ((data["price_per_bed"] > (data["common_roof_deck"] * (data["bedrooms"] - ((data["manager_id_mean_high"] > ((data["patio"] > data["building_id_mean_high"]).astype(float))).astype(float))))).astype(float)))))))))).astype(float)))))
    v["i55"] = np.tanh(((data["common_roof_deck"] * ((((((data["dogs_allowed"] * (data["bedrooms"] * data["outdoor_space"])) + data["business_center"])/2.0) + ((((data["dogs_allowed"] < (data["manager_level_high"] - (data["fitness_center"] * (((data["outdoor_space"] + data["no_fee"])/2.0) * (data["outdoor_space"] * 2.0))))).astype(float)) > data["manager_level_high"]).astype(float)))/2.0) * (-((data["no_fee"] * 2.0))))) * 2.0))
    v["i56"] = np.tanh((data["dishwasher"] * (-((data["work"] * (((((((-((((data["garage"] - (data["manager_id"] - data["wifi_access"])) < ((data["num_photos"] < (data["dishwasher"] - data["work"])).astype(float))).astype(float)))) * 2.0) + (((data["num_photos"] + data["hardwood_floors"])/2.0) * 2.0))/2.0) * 2.0) - data["doorman"]) - data["parking_space"]))))))
    v["i57"] = np.tanh((((((data["dining_room"] * data["microwave"]) - (data["washer_in_unit"] * data["in_superintendent"])) + (data["microwave"] - (data["site_parking_lot"] * data["high_ceiling"])))/2.0) * ((data["building_id_mean_high"] - (data["dogs_allowed"] * data["building_id_mean_medium"])) - (((data["high_ceiling"] * (data["dogs_allowed"] * data["building_id_mean_medium"])) < data["dining_room"]).astype(float)))))
    v["i58"] = np.tanh(((data["work"] * 2.0) * (((data["live"] - (data["balcony"] * data["hardwood_floors"])) / 2.0) - ((data["laundry_in_building"] + data["laundry_in_building"]) * (((data["hardwood_floors"] + ((data["manager_level_medium"] + ((data["work"] * 2.0) + (data["manager_level_medium"] / 2.0))) * data["manager_level_medium"])) < data["war"]).astype(float))))))
    v["i59"] = np.tanh((data["common_roof_deck"] * ((((data["price_per_room"] > data["roof"]).astype(float)) + (data["price_per_room"] - ((((data["level"] * data["building_id_mean_medium"]) * data["building_id_mean_medium"]) + ((data["building_id_mean_medium"] > data["laundry_in_building"]).astype(float)))/2.0))) + (((data["manager_id_mean_high"] * data["building_id_mean_medium"]) + data["laundry_in_building"]) * ((data["price_per_room"] < (data["level"] * data["building_id_mean_medium"])).astype(float))))))
    v["i60"] = np.tanh((-(((np.tanh(np.tanh((((data["price"] + ((-((((((data["high_ceilings"] + data["manager_level_low"])/2.0) / 2.0) < (data["high_ceilings"] + (data["longitude"] - (-((((((data["manager_level_low"] + (data["furnished"] * data["price"]))/2.0) / 2.0) < data["high_ceilings"]).astype(float))))))).astype(float)))) * data["manager_level_low"]))/2.0) / 2.0))) < data["time_doorman"]).astype(float)))))
    v["i61"] = np.tanh((data["fireplace"] * ((((data["_photos"] * ((data["private_outdoor_space"] * data["building_id_mean_medium"]) * 2.0)) + (data["parking"] * ((data["building_id_mean_medium"] < data["private_outdoor_space"]).astype(float)))) + ((data["storage"] > ((data["wifi_access"] + (-((data["latitude"] + ((data["high_ceiling"] > data["_pets_ok_"]).astype(float))))))/2.0)).astype(float)))/2.0)))
    v["i62"] = np.tanh(((data["space"] * (-((data["building_id"] - ((data["short_term_allowed"] * (-((data["building_id"] - data["num_description_words"])))) + ((data["dining_room"] - (data["num_description_words"] * ((data["building_id"] + data["actual_apt"])/2.0))) + (data["private_outdoor_space"] - ((data["prewar"] < (data["building_id"] * data["num_description_words"])).astype(float))))))))) / 2.0))
    v["i63"] = np.tanh((((((data["residents_lounge"] * ((data["roof_deck"] - (((((((data["pre"] > data["manager_level_high"]).astype(float)) * 0.318310) > data["manager_level_high"]).astype(float)) * (data["duplex"] + data["short_term_allowed"])) + data["space"])) * 2.0)) > (-(data["space"]))).astype(float)) > (-((((data["newly_renovated"] > data["duplex"]).astype(float)) * data["pre"])))).astype(float)))
    v["i64"] = np.tanh((np.tanh(data["laundry_in_unit"]) * (-((data["high_ceiling"] * (data["washer"] - ((data["parking_space"] + (data["price_per_room"] + (data["washer"] + (data["granite_kitchen"] + (data["price_per_room"] + data["parking_space"]))))) * ((data["price_per_room"] > ((data["laundry_in_unit"] > (data["high_ceiling"] + ((data["washer"] + data["granite_kitchen"])/2.0))).astype(float))).astype(float)))))))))
    v["i65"] = np.tanh((np.tanh(data["pool"]) * (((data["longitude"] > data["latitude"]).astype(float)) - (data["furnished"] * ((((np.tanh(data["latitude"]) + data["fitness_center"])/2.0) - data["no_fee"]) * ((data["fitness_center"] < (((data["latitude"] < (((data["furnished"] < data["no_fee"]).astype(float)) / 2.0)).astype(float)) - data["no_fee"])).astype(float)))))))
    v["i66"] = np.tanh((data["health_club"] + (-(((((data["price_per_room"] + (data["price_per_bath"] + ((data["price_per_bath"] + ((data["price_per_room"] + np.tanh((data["price_per_bath"] + data["price_per_bed"])))/2.0))/2.0)))/2.0) + (((-(((0.318310 + (data["microwave"] * ((data["exclusive"] + ((data["price_per_room"] + data["air_conditioning"])/2.0))/2.0)))/2.0))) > data["price_per_bath"]).astype(float)))/2.0)))))
    v["i67"] = np.tanh((((data["private_balcony"] * 2.0) * ((np.tanh(((data["bathrooms"] > (-((((((-(data["manager_level_high"])) > ((data["manager_level_medium"] * np.tanh(((data["manager_level_medium"] * data["fitness_center"]) * 2.0))) * 2.0)).astype(float)) < (-(np.tanh(data["doorman"])))).astype(float))))).astype(float))) > ((np.tanh(data["doorman"]) < (-(data["manager_level_high"]))).astype(float))).astype(float))) * 2.0))
    v["i68"] = np.tanh((data["eat_in_kitchen"] * ((((data["listing_id"] * (data["building_id"] + (data["listing_id"] * ((data["light"] + data["listing_id"])/2.0)))) > data["manager_id_mean_high"]).astype(float)) - ((data["manager_id_mean_high"] > (data["building_id"] + ((data["building_id"] > (((((data["manager_id_mean_high"] > data["price"]).astype(float)) + data["eat_in_kitchen"]) > data["listing_id"]).astype(float))).astype(float)))).astype(float)))))
    v["i69"] = np.tanh((data["pets_on_approval"] * ((data["duplex"] + ((-(((((((((((data["manager_level_medium"] > data["pets_on_approval"]).astype(float)) / 2.0) > data["building_id"]).astype(float)) * data["swimming_pool"]) / 2.0) > data["building_id"]).astype(float)) * data["swimming_pool"]))) * ((((((data["manager_level_medium"] + (data["building_id"] * data["manager_id_mean_high"]))/2.0) + data["pre"])/2.0) * 2.0) * 2.0)))/2.0)))
    v["i70"] = np.tanh((-(((data["residents_garden"] > (-((data["bathrooms"] * (((data["bathrooms"] - (((data["_photos"] < ((data["_photos"] < ((data["bathrooms"] < data["bathrooms"]).astype(float))).astype(float))).astype(float)) * 2.0)) < (data["_photos"] - ((data["washer_"] < np.tanh(((data["laundry_room"] - (data["bedrooms"] - data["central_ac"])) * data["health_club"]))).astype(float)))).astype(float)))))).astype(float)))))
    v["i71"] = np.tanh((np.tanh(((data["short_term_allowed"] * 2.0) * 2.0)) * (data["latitude"] - ((data["manager_level_low"] > (data["bedrooms"] * ((((data["bedrooms"] * data["cats_allowed"]) - ((data["bedrooms"] < ((3.141593 < data["manager_level_low"]).astype(float))).astype(float))) - ((((data["manager_level_low"] > (data["bedrooms"] * data["cats_allowed"])).astype(float)) > data["bedrooms"]).astype(float))) / 2.0))).astype(float)))))
    v["i72"] = np.tanh(((data["actual_apt"] * 2.0) * ((((((-(((data["reduced_fee"] + ((((((data["newly_renovated"] + ((data["newly_renovated"] + data["exclusive"])/2.0))/2.0) + ((data["manager_id_mean_high"] < data["pre"]).astype(float)))/2.0) * 2.0) * data["dogs_allowed"]))/2.0))) * 2.0) + ((data["dogs_allowed"] < data["pre"]).astype(float)))/2.0) > np.tanh(data["manager_id_mean_high"])).astype(float))))
    v["i73"] = np.tanh(((data["price"] * ((((data["storage"] > data["building_id_mean_high"]).astype(float)) - (((data["building_id_mean_high"] * data["bedrooms"]) < data["price"]).astype(float))) - (((data["building_id_mean_medium"] * (data["bedrooms"] * 2.0)) < data["storage"]).astype(float)))) * ((data["private_backyard"] > ((data["building_id_mean_medium"] + (-(((data["price_per_bath"] < data["stainless_steel_appliances"]).astype(float)))))/2.0)).astype(float))))
    v["i74"] = np.tanh(((data["residents_lounge"] + (data["residents_lounge"] * (-(((data["dishwasher"] + data["bedrooms"]) + (data["num_description_words"] - (data["laundry_in_building"] * (-((((data["elevator"] + ((((data["dishwasher"] < data["bedrooms"]).astype(float)) + data["high_ceiling"])/2.0)) + data["manager_id"]) + ((data["green_building"] + data["green_building"])/2.0)))))))))))/2.0))
    v["i75"] = np.tanh(((data["concierge"] * 2.0) * (((((data["war"] > data["concierge"]).astype(float)) * 2.0) < ((np.tanh(data["war"]) - (((-(((data["num_description_words"] > ((((data["manager_level_low"] - (-(data["dishwasher"]))) < data["view"]).astype(float)) / 2.0)).astype(float)))) > data["num_photos"]).astype(float))) - (data["view"] - (-(data["num_photos"]))))).astype(float))))
    v["i76"] = np.tanh((data["children"] * (((data["pool"] * 2.0) + (data["storage"] + ((data["valet"] - (data["walk_in_closet"] - (data["latitude"] - data["duplex"]))) / 2.0))) + ((data["common_parking"] > (-(np.tanh(((data["latitude"] + ((((data["longitude"] > data["latitude"]).astype(float)) > (data["duplex"] - data["pre"])).astype(float)))/2.0))))).astype(float)))))
    v["i77"] = np.tanh((-((data["price"] * ((data["price_per_bath"] < (data["furnished"] + np.tanh((((data["price_per_bath"] * (((data["manager_level_low"] - data["listing_id"]) * 2.0) + data["listing_id"])) < (data["furnished"] + ((data["listing_id"] + (-(((data["listing_id"] + data["price_per_bath"])/2.0))))/2.0))).astype(float))))).astype(float))))))
    v["i78"] = np.tanh((data["virtual_doorman"] * (((data["high_speed_internet"] - np.tanh(data["in_super"])) - (-(((((data["building_id_mean_high"] * data["in_super"]) * 2.0) * (data["building_id_mean_high"] * np.tanh(data["building_id_mean_high"]))) * 2.0)))) * (-(((((data["no_fee"] < np.tanh(np.tanh((data["duplex"] - data["building_id_mean_high"])))).astype(float)) * 2.0) * 2.0))))))
    v["i79"] = np.tanh((-((data["flex"] + (data["private"] * ((((data["balcony"] + data["exclusive"]) + ((data["site_super"] - data["site_super"]) - data["num_photos"])) * (((data["bedrooms"] > data["speed_internet"]).astype(float)) + ((data["exclusive"] < data["flex"]).astype(float)))) * ((data["latitude"] < (-((data["listing_id"] * data["private"])))).astype(float))))))))
    v["i80"] = np.tanh((((data["price"] + (data["space"] * ((data["marble_bath"] + ((data["building_id"] > data["children"]).astype(float))) + ((data["dining_room"] + ((data["building_id"] > (data["price_per_bath"] * 2.0)).astype(float))) + ((data["highrise"] > (data["price"] * 2.0)).astype(float))))))/2.0) * (data["price"] + ((data["price"] > (data["highrise"] * 2.0)).astype(float)))))
    v["i81"] = np.tanh((((-((data["concierge"] * ((data["garage"] > (((((data["concierge"] + data["building_id_mean_high"]) + ((data["elevator"] < (((data["wheelchair_access"] + data["building_id_mean_medium"]) + data["wheelchair_access"])/2.0)).astype(float)))/2.0) + (data["midrise"] + ((data["doorman"] < (data["wheelchair_access"] - ((data["new_construction"] + data["garage"])/2.0))).astype(float))))/2.0)).astype(float))))) * 2.0) * 2.0))
    v["i82"] = np.tanh(((data["unit_washer"] > (-(((data["exclusive"] * ((((data["furnished"] + data["price"])/2.0) > (((data["furnished"] * data["laundry_in_unit"]) + data["longitude"])/2.0)).astype(float))) + (data["longitude"] * ((data["war"] < ((data["fireplace"] - data["laundry_in_unit"]) - ((data["fireplace"] < ((data["furnished"] + data["price"])/2.0)).astype(float)))).astype(float))))))).astype(float)))
    v["i83"] = np.tanh((data["roof"] * (((((data["latitude"] + data["no_pets"])/2.0) > data["longitude"]).astype(float)) * (((data["fireplace"] * (-(data["duplex"]))) < (np.tanh((data["longitude"] * ((data["latitude"] < ((((data["longitude"] > data["wheelchair_ramp"]).astype(float)) + (-(data["longitude"])))/2.0)).astype(float)))) * 2.0)).astype(float)))))
    v["i84"] = np.tanh((-(((data["gym_in_building"] + (data["outdoor_areas"] - (data["balcony"] * (-((data["level"] * (data["bedrooms"] * ((data["reduced_fee"] > ((data["building_id_mean_medium"] + data["short_term_allowed"]) * data["publicoutdoor"])).astype(float))))))))) * (((data["balcony"] * data["balcony"]) < (data["bedrooms"] + data["building_id_mean_medium"])).astype(float))))))
    v["i85"] = np.tanh(((((data["granite_kitchen"] * ((((data["latitude"] + data["longitude"])/2.0) > data["_photos"]).astype(float))) + data["fitness"]) + (data["private"] * data["longitude"])) * ((data["business_center"] > ((data["latitude"] + (data["fitness"] * ((data["longitude"] > ((data["granite_kitchen"] + ((data["_photos"] < data["fitness"]).astype(float)))/2.0)).astype(float))))/2.0)).astype(float))))
    v["i86"] = np.tanh((data["latitude"] * (-(((-((data["manager_id_mean_medium"] + np.tanh((data["manager_id_mean_high"] - (np.tanh((data["manager_id_mean_high"] * data["manager_id_mean_high"])) * 2.0)))))) + ((data["manager_id_mean_high"] + (data["common_outdoor_space"] - np.tanh((((data["manager_id"] > data["manager_id_mean_medium"]).astype(float)) - ((data["manager_id_mean_high"] * data["manager_id_mean_medium"]) * 2.0)))))/2.0))))))
    v["i87"] = np.tanh((-(((data["wifi_access"] > (-(((data["deck"] > ((data["price_per_bath"] + (-((data["price_per_bath"] * (((data["live"] + ((data["furnished"] + ((((data["price_per_bath"] + (data["deck"] + ((data["price_per_bath"] + ((data["bathrooms"] + data["residents_garden"])/2.0))/2.0)))/2.0) + data["parking_space"])/2.0))/2.0))/2.0) * 2.0)))))/2.0)).astype(float))))).astype(float)))))
    v["i88"] = np.tanh(((data["price_per_room"] + (-((data["price_per_room"] * ((data["num_photos"] > ((-((((data["bathrooms"] + data["common_outdoor_space"])/2.0) + (data["price_per_room"] * data["num_photos"])))) / 2.0)).astype(float)))))) * ((data["site_garage"] + ((data["pre"] * data["num_photos"]) + data["bathrooms"])) + (((data["marble_bath"] + data["pre"])/2.0) / 2.0))))
    v["i89"] = np.tanh((data["manager_id"] * ((data["outdoor_areas"] * (np.tanh(np.tanh((data["building_id"] + (data["dogs_allowed"] - ((((data["walk"] > data["building_id_mean_medium"]).astype(float)) > data["listing_id"]).astype(float)))))) - ((data["private_outdoor_space"] > (data["building_id_mean_medium"] + ((np.tanh(data["dogs_allowed"]) < ((data["listing_id"] > ((data["private_terrace"] > data["listing_id"]).astype(float))).astype(float))).astype(float)))).astype(float)))) * 2.0)))
    v["i90"] = np.tanh(((data["price_per_bed"] > ((1.0 + (data["doorman"] * ((data["gym_in_building"] * 2.0) + np.tanh(((data["price_per_bed"] + (data["furnished"] * ((data["manager_level_low"] + ((((data["manager_level_low"] + ((data["price_per_bed"] < data["doorman"]).astype(float)))/2.0) < ((data["site_laundry"] + np.tanh(((data["dryer_in_unit"] < data["doorman"]).astype(float))))/2.0)).astype(float)))/2.0))) * 2.0)))))/2.0)).astype(float)))
    v["i91"] = np.tanh(((-(((((data["furnished"] + (data["doorman"] * ((((data["doorman"] * (data["loft"] - ((data["furnished"] + data["doorman"])/2.0))) < data["manager_level_low"]).astype(float)) - data["furnished"])))/2.0) + (-(data["lounge"])))/2.0))) * ((data["indoor_pool"] > (data["common_roof_deck"] * data["reduced_fee"])).astype(float))))
    v["i92"] = np.tanh(((((data["price"] * ((data["private_terrace"] + (data["price"] - (data["num_description_words"] * ((data["manager_level_low"] > data["short_term_allowed"]).astype(float))))) - (data["num_description_words"] * data["manager_level_low"]))) + ((-((((((data["price"] > (data["pet_friendly"] / 2.0)).astype(float)) + data["short_term_allowed"]) > ((data["num_description_words"] > data["sauna"]).astype(float))).astype(float)))) / 2.0))/2.0) / 2.0))
    v["i93"] = np.tanh((-((data["residents_lounge"] * ((((-(data["listing_id"])) * ((((data["price_per_bed"] > (data["space"] * data["listing_id"])).astype(float)) + ((((data["price_per_bath"] > (-(((data["furnished"] + ((data["furnished"] > (data["listing_id"] * (-(data["price_per_bath"])))).astype(float)))/2.0)))).astype(float)) > (-(data["furnished"]))).astype(float)))/2.0)) * 2.0) * 2.0)))))
    v["i94"] = np.tanh((data["furnished"] * (data["fitness"] * (np.tanh((((np.tanh((np.tanh(data["building_id_mean_high"]) + data["building_id_mean_high"])) + ((data["manager_id"] + data["cats_allowed"])/2.0)) * 2.0) * 2.0)) + (((-(((data["fitness"] > data["building_id_mean_high"]).astype(float)))) + np.tanh((data["manager_level_medium"] + data["building_id_mean_high"])))/2.0)))))
    v["i95"] = np.tanh((data["longitude"] * (-1.0 - ((((data["no_fee"] + (data["lounge_room"] - data["furnished"])) * data["patio"]) + (data["furnished"] - (((data["patio"] + ((data["lounge_room"] - data["virtual_doorman"]) + data["manager_id_mean_medium"])) - (data["private_terrace"] + data["no_fee"])) - (data["washer"] + data["private_terrace"]))))/2.0))))
    v["i96"] = np.tanh(((data["sauna"] * (((((data["bedrooms"] - data["manager_level_medium"]) > np.tanh((((data["bedrooms"] * 2.0) > (((data["sauna"] + ((data["no_pets"] * data["manager_level_medium"]) * data["listing_id"]))/2.0) / 2.0)).astype(float)))).astype(float)) < (data["site_garage"] * (-((((data["midrise"] + (data["no_pets"] * data["listing_id"]))/2.0) / 2.0))))).astype(float))) * 2.0))
    v["i97"] = np.tanh(((((-(data["terrace"])) < (((((-(data["terrace"])) / 2.0) / 2.0) < (((data["latitude"] * (-(((data["terrace"] < ((data["outdoor_space"] + ((-(data["latitude"])) / 2.0))/2.0)).astype(float))))) < (data["in_super"] * 2.0)).astype(float))).astype(float))).astype(float)) * ((((data["outdoor_space"] + ((-(data["terrace"])) / 2.0))/2.0) / 2.0) / 2.0)))
    v["i98"] = np.tanh((-((data["wifi_access"] * ((data["price_per_bath"] < ((data["wifi_access"] + (data["dryer"] + (((data["dryer"] + ((np.tanh(data["price_per_bath"]) > ((data["decorative_fireplace"] * ((data["price_per_bath"] < ((data["fitness"] + (data["brownstone"] + ((((data["fitness"] < data["decorative_fireplace"]).astype(float)) < data["manager_level_medium"]).astype(float))))/2.0)).astype(float))) * 2.0)).astype(float)))/2.0) / 2.0)))/2.0)).astype(float))))))
    v["i99"] = np.tanh(((data["roof"] * (((data["longitude"] < data["parking"]).astype(float)) + (((data["num_photos"] * data["walk"]) < ((data["longitude"] < data["parking"]).astype(float))).astype(float)))) * (((((data["parking"] < data["longitude"]).astype(float)) < np.tanh(data["bedrooms"])).astype(float)) + ((data["num_photos"] < ((data["walk"] < data["longitude"]).astype(float))).astype(float)))))

    return Outputs(v.sum(axis=1))


if __name__ == "__main__":
    myseed=321
    random.seed(myseed)
    np.random.seed(myseed)
    X_train = pd.read_json("../input/train.json")
    X_test = pd.read_json("../input/test.json")
    
    index=list(range(X_train.shape[0]))
    random.shuffle(index)
    a=[np.nan]*len(X_train)
    b=[np.nan]*len(X_train)
    c=[np.nan]*len(X_train)

    for i in range(5):
        building_level={}
        for j in X_train['manager_id'].values:
            building_level[j]=[0,0,0]
        test_index=index[int((i*X_train.shape[0])/5):int(((i+1)*X_train.shape[0])/5)]
        train_index=list(set(index).difference(test_index))
        for j in train_index:
            temp=X_train.iloc[j]
            if temp['interest_level']=='low':
                building_level[temp['manager_id']][0]+=1
            if temp['interest_level']=='medium':
                building_level[temp['manager_id']][1]+=1
            if temp['interest_level']=='high':
                building_level[temp['manager_id']][2]+=1
        for j in test_index:
            temp=X_train.iloc[j]
            if sum(building_level[temp['manager_id']])!=0:
                a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
                b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
                c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
    X_train['manager_level_low']=a
    X_train['manager_level_medium']=b
    X_train['manager_level_high']=c

    a=[]
    b=[]
    c=[]
    building_level={}
    for j in X_train['manager_id'].values:
        building_level[j]=[0,0,0]
    for j in range(X_train.shape[0]):
        temp=X_train.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1

    for i in X_test['manager_id'].values:
        if i not in building_level.keys():
            a.append(np.nan)
            b.append(np.nan)
            c.append(np.nan)
        else:
            a.append(building_level[i][0]*1.0/sum(building_level[i]))
            b.append(building_level[i][1]*1.0/sum(building_level[i]))
            c.append(building_level[i][2]*1.0/sum(building_level[i]))
    X_test['manager_level_low']=a
    X_test['manager_level_medium']=b
    X_test['manager_level_high']=c
    interest_level_map = {'low': 0, 'medium': 1, 'high': 2}
    X_train['interest_level'] = X_train['interest_level'].apply(lambda x: interest_level_map[x])
    X_test['interest_level'] = -1
    X_train['created'] = pd.to_datetime(X_train["created"])
    X_test['created'] = pd.to_datetime(X_test["created"])
    feature_transform = CountVectorizer(stop_words='english', max_features=150)
    X_train['features'] = X_train["features"].apply(lambda x: " ".join(["_".join(i.lower().split(" ")) for i in x]))
    X_test['features'] = X_test["features"].apply(lambda x: " ".join(["_".join(i.lower().split(" ")) for i in x]))
    feature_transform.fit(list(X_train['features']) + list(X_test['features']))

    train_size = len(X_train)
    low_count = len(X_train[X_train['interest_level'] == 0])
    medium_count = len(X_train[X_train['interest_level'] == 1])
    high_count = len(X_train[X_train['interest_level'] == 2])

    managers_with_one_lot = find_objects_with_only_one_record('manager_id')
    buildings_with_one_lot = find_objects_with_only_one_record('building_id')
    addresses_with_one_lot = find_objects_with_only_one_record('display_address')

    print("Starting transformations")
    X_train = transform_data(X_train)
    X_test = transform_data(X_test)
    y = X_train['interest_level'].ravel()
    testids = X_test['listing_id'].ravel()
    print("Normalizing high cardinality data...")
    normalize_high_cardinality_data()
    transform_categorical_data()
    remove_columns(X_train)
    remove_columns(X_test)
    X_train.replace(np.inf, np.nan, inplace=True)
    X_test.replace(np.inf, np.nan, inplace=True)
    X_train.fillna(-1, inplace=True)
    X_test.fillna(-1, inplace=True)
    features = X_train.columns
    ss = StandardScaler()
    alldata = pd.concat([X_train[features], X_test[features]])
    ss.fit(alldata)
    scaledtrain = X_train.copy()
    scaledtest = X_test.copy()
    scaledtrain[features] = ss.transform(scaledtrain[features])
    scaledtest[features] = ss.transform(scaledtest[features])
    scaledtrain = scaledtrain[features]
    scaledtest = scaledtest[features]
    scaledtrain['interest_level'] = y.astype(int)
    scaledtest = scaledtest[features]
    scaledtest.insert(0, 'realids', testids)
    subtrain = pd.DataFrame()
    subtrain['low'] = GPZero(scaledtrain)
    subtrain['medium'] = GPOne(scaledtrain)
    subtrain['high'] = GPTwo(scaledtrain)
    subtrain['interest_level'] = scaledtrain.interest_level.ravel()
    print(log_loss(subtrain['interest_level'],subtrain[['low', 'medium', 'high']]))
    subtest = pd.DataFrame()
    subtest['listing_id'] = scaledtest.realids.values
    subtest['low'] = GPZero(scaledtest)
    subtest['medium'] = GPOne(scaledtest)
    subtest['high'] = GPTwo(scaledtest)
    subtest.to_csv('gpsubmission.csv', index=False)

Starting transformations
Normalizing high cardinality data...
0.522981113402


In [23]:
data_path = "../input/"
train_X_0322 = pd.read_csv(data_path + 'train_BM_MB_add03052240.csv')
test_X_0322 = pd.read_csv(data_path + 'test_BM_MB_add03052240.csv')

print train_X_0322.shape
print test_X_0322.shape

(49352, 322)
(74659, 322)


In [31]:
subtrain["listing_id"] = X_train.listing_id.values

In [36]:
subtrain.head()

low    medium      high  interest_level  listing_id
0  0.871032  0.132950  0.002816               1     7211212
1  0.995209  0.017996  0.000796               0     7150865
2  0.448348  0.389290  0.075313               2     6887163
3  0.922767  0.085197  0.001626               0     6888711
4  0.988413  0.019866  0.003426               0     6934781

In [34]:
tmp_train = train_X_0322[['listing_id']].merge(subtrain,on = 'listing_id', how = 'left')[["low", "medium", "high"]].values
tmp_test_mean = test_X_0322[['listing_id']].merge(subtest,on = 'listing_id', how = 'left')[["low", "medium", "high"]].values

In [37]:
tmp_test_mean

array([[  4.63655662e-01,   3.98541376e-01,   6.41895065e-02],
       [  9.50985566e-01,   5.43754920e-02,   3.28965538e-02],
       [  9.28136480e-01,   8.37766705e-02,   9.27267812e-03],
       ..., 
       [  9.73060982e-01,   3.51015863e-02,   6.07826922e-03],
       [  9.89801541e-01,   1.17846666e-02,   2.81515324e-04],
       [  5.83356148e-01,   3.65695802e-01,   3.81721970e-02]])

In [38]:
from datetime import datetime
now = datetime.now()

name_train_blend = '../output/train_blend_GP_BM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../output/test_blend_GP_BM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

np.savetxt(name_train_blend,tmp_train, delimiter=",")
np.savetxt(name_test_blend_mean,tmp_test_mean, delimiter=",")

In [41]:
tmp_test_mean_sub = test_X_0322[['listing_id']].merge(subtest,on = 'listing_id', how = 'left')[["low", "medium", "high",'listing_id']]
tmp_test_mean_sub.to_csv('gpsubmission.csv', index=False)

In [43]:
tmp_test_mean_sub.head()

low    medium      high  listing_id
0  0.463656  0.398541  0.064190     7142618
1  0.950986  0.054375  0.032897     7210040
2  0.928136  0.083777  0.009273     7174566
3  0.289910  0.419127  0.115973     7191391
4  0.814114  0.182908  0.016571     7171695